# Soru 1: Tüm verileri göster

In [1]:
import pandas as pd 
import sqlite3
con = sqlite3.connect("sqlite-sakila.db")


In [2]:

# Tabloyu içeri al (bu adım sadece tabloyu çekmek için, sorgu çözümü değil)
film = pd.read_sql("SELECT * FROM film;", con)
film.head()

,film_id,title,description,release_year,language_id,original_language_id,rental_duration,rental_rate,length,replacement_cost,rating,special_features,last_update
0,1,ACADEMY DINOSAUR,A Epic Drama of a Feminist And a Mad Scientist...,2006,1,None,6,0.99,86,20.99,PG,"Deleted Scenes,Behind the Scenes",2021-03-06 15:52:00
1,2,ACE GOLDFINGER,A Astounding Epistle of a Database Administrat...,2006,1,None,3,4.99,48,12.99,G,"Trailers,Deleted Scenes",2021-03-06 15:52:00
2,3,ADAPTATION HOLES,A Astounding Reflection of a Lumberjack And a ...,2006,1,None,7,2.99,50,18.99,NC-17,"Trailers,Deleted Scenes",2021-03-06 15:52:00
3,4,AFFAIR PREJUDICE,A Fanciful Documentary of a Frisbee And a Lumb...,2006,1,None,5,2.99,117,26.99,G,"Commentaries,Behind the Scenes",2021-03-06 15:52:00
4,5,AFRICAN EGG,A Fast-Paced Documentary of a Pastry Chef And ...,2006,1,None,6,2.99,130,22.99,G,Deleted Scenes,2021-03-06 15:52:00


# Soru 2: Her filmdeki oyuncuları listele

In [3]:
# Önce gerekli tabloları al
film       = pd.read_sql("SELECT * FROM film;", con)
actor      = pd.read_sql("SELECT * FROM actor;", con)
film_actor = pd.read_sql("SELECT * FROM film_actor;", con)

# Oyuncu adını birleştir
actor2 = actor.assign(actor_name = actor["first_name"] + " " + actor["last_name"])[
    ["actor_id","actor_name"]
]

# Join işlemleri
fa_names = (film_actor
            .merge(actor2, on="actor_id", how="inner")
            .merge(film[["film_id","title"]], on="film_id", how="inner"))

# Film bazında oyuncu listesi (liste formatında)
pd_s2_list = (fa_names
              .sort_values(["film_id","actor_name"])
              .groupby(["film_id","title"], as_index=False)["actor_name"]
              .agg(list))

# SQL çıktısına benzetmek için virgül ile birleştirme
pd_s2 = pd_s2_list.assign(
    actors = pd_s2_list["actor_name"].apply(lambda L: ", ".join(L))
)[["film_id","title","actors"]]

pd_s2.head()


,film_id,title,actors
0,1,ACADEMY DINOSAUR,"CHRISTIAN GABLE, JOHNNY CAGE, LUCILLE TRACY, M..."
1,2,ACE GOLDFINGER,"BOB FAWCETT, CHRIS DEPP, MINNIE ZELLWEGER, SEA..."
2,3,ADAPTATION HOLES,"BOB FAWCETT, CAMERON STREEP, JULIANNE DENCH, N..."
3,4,AFFAIR PREJUDICE,"FAY WINSLET, JODIE DEGENERES, KENNETH PESCI, O..."
4,5,AFRICAN EGG,"DUSTIN TAUTOU, GARY PHOENIX, MATTHEW CARREY, M..."


# Soru 3: Her filmde kaç oyuncu oynadı?


In [4]:
# 1) Yalnız tabloları içeri al
film       = pd.read_sql("SELECT film_id, title FROM film;", con)
film_actor = pd.read_sql("SELECT film_id, actor_id FROM film_actor;", con)

# 2) Agregasyon: film bazında oyuncu sayısı
fa_counts = (film_actor
             .groupby("film_id", as_index=False)
             .agg(actor_count=("actor_id","nunique")))

# 3) Başlık ekle ve sırala
pd_s3 = (film.merge(fa_counts, on="film_id", how="inner")
              .sort_values("film_id")
              .reset_index(drop=True))

pd_s3.head()

,film_id,title,actor_count
0,1,ACADEMY DINOSAUR,10
1,2,ACE GOLDFINGER,4
2,3,ADAPTATION HOLES,5
3,4,AFFAIR PREJUDICE,5
4,5,AFRICAN EGG,5


# Soru 4: Her oyuncu kaç filmde oynadı?

In [5]:
# 1) Tabloları oku
actor      = pd.read_sql("SELECT actor_id, first_name, last_name FROM actor;", con)
film_actor = pd.read_sql("SELECT film_id, actor_id FROM film_actor;", con)

# 2) Oyuncu isimlerini birleştir
actor = actor.assign(actor_name = actor["first_name"] + " " + actor["last_name"])

# 3) Agregasyon: her oyuncunun film sayısı
film_counts = (film_actor
               .groupby("actor_id", as_index=False)
               .agg(film_count=("film_id","nunique")))

# 4) Sonuç
pd_s4 = (actor.merge(film_counts, on="actor_id", how="inner")
               .sort_values("film_count", ascending=False)
               .reset_index(drop=True))

pd_s4.head()


,actor_id,first_name,last_name,actor_name,film_count
0,107,GINA,DEGENERES,GINA DEGENERES,42
1,102,WALTER,TORN,WALTER TORN,41
2,198,MARY,KEITEL,MARY KEITEL,40
3,181,MATTHEW,CARREY,MATTHEW CARREY,39
4,23,SANDRA,KILMER,SANDRA KILMER,37


# Soru 5: Envanterde olmayan filmler var mı ve varsa kaç tane?

In [6]:
# 1) Tabloları oku
film     = pd.read_sql("SELECT film_id, title FROM film;", con)
inventory= pd.read_sql("SELECT inventory_id, film_id FROM inventory;", con)

# 2) film ile inventory sol join
merged = film.merge(inventory, on="film_id", how="left")

# 3) envanterde olmayan filmler
missing_films = merged[merged["inventory_id"].isna()]

# 4) kaç tane?
missing_film_count = len(missing_films)
print("Envanterde olmayan film sayısı:", missing_film_count)


Envanterde olmayan film sayısı: 42


#  Soru 6: Kiralanabilir olan her filmin kaç kez kiralandığını ve toplam gelirlerini getirin


In [7]:
# 1) Tabloları al
film     = pd.read_sql("SELECT film_id, title FROM film;", con)
inventory= pd.read_sql("SELECT inventory_id, film_id FROM inventory;", con)
rental   = pd.read_sql("SELECT rental_id, inventory_id FROM rental;", con)
payment  = pd.read_sql("SELECT payment_id, rental_id, amount FROM payment;", con)

# 2) Join işlemleri
merged = (film
          .merge(inventory, on="film_id", how="inner")
          .merge(rental, on="inventory_id", how="inner")
          .merge(payment, on="rental_id", how="inner"))

# 3) Agregasyon: kiralanma sayısı ve toplam gelir
pd_s6 = (merged
         .groupby(["film_id","title"], as_index=False)
         .agg(rental_count=("rental_id","count"),
              total_revenue=("amount","sum"))
         .sort_values("rental_count", ascending=False))

pd_s6.head()


,film_id,title,rental_count,total_revenue
96,103,BUCKET BROTHERHOOD,34,180.66
705,738,ROCKETEER MOTHER,33,116.67
733,767,SCALAWAG DUCK,32,172.68
697,730,RIDGEMONT SUBMARINE,32,130.68
361,382,GRIT CLOCKWORK,32,110.68


# Soru 7: Envanterde olmayan filmlerin kira oranlarını getirin

In [8]:
# 1) Tabloları al
film      = pd.read_sql("SELECT film_id, title, rental_rate FROM film;", con)
inventory = pd.read_sql("SELECT inventory_id, film_id FROM inventory;", con)

# 2) Left join
merged = film.merge(inventory, on="film_id", how="left")

# 3) Envanterde olmayanları filtrele
missing_films = merged[merged["inventory_id"].isna()][["film_id","title","rental_rate"]]

missing_films.head()


,film_id,title,rental_rate
70,14,ALICE FANTASIA,0.99
156,33,APOLLO TEEN,2.99
168,36,ARGONAUTS TOWN,0.99
176,38,ARK RIDGEMONT,0.99
188,41,ARSENIC INDEPENDENCE,0.99


# Soru 8: Birden fazla DVD'yi iade etmeyen kaç müşteri var?

In [9]:
# 1) rental tablosunu al
rental = pd.read_sql("SELECT rental_id, customer_id, return_date FROM rental;", con)

# 2) İade edilmeyen kiralamaları filtrele
not_returned = rental[rental["return_date"].isna()]

# 3) Müşteri bazında iade edilmeyen sayısı
cust_counts = (not_returned
               .groupby("customer_id", as_index=False)
               .agg(not_returned=("rental_id","count")))

# 4) Birden fazla iade edilmeyen müşterileri filtrele
multi_not_returned = cust_counts[cust_counts["not_returned"] > 1]

# 5) Kaç müşteri?
customer_count = len(multi_not_returned)
print("Birden fazla DVD’yi iade etmeyen müşteri sayısı:", customer_count)


Birden fazla DVD’yi iade etmeyen müşteri sayısı: 23


# Soru 9: Her müşteri kaç film kiraladı?

In [10]:
# Yalnız tablo çekme
rental    = pd.read_sql("SELECT rental_id, customer_id, inventory_id FROM rental;", con)
inventory = pd.read_sql("SELECT inventory_id, film_id FROM inventory;", con)

# (1) Toplam kiralama sayısı
pd_total = (rental
            .groupby("customer_id", as_index=False)
            .agg(total_rentals=("rental_id","count"))
            .sort_values("customer_id"))

pd_total.head()


,customer_id,total_rentals
0,1,32
1,2,27
2,3,26
3,4,22
4,5,38


# Soru 10: Türlerine göre en çok kiralanan filmler ve bunlara ne kadar ödendi?

In [11]:

# 1) Tabloları çek
category      = pd.read_sql("SELECT category_id, name FROM category;", con)
film_category = pd.read_sql("SELECT film_id, category_id FROM film_category;", con)
film          = pd.read_sql("SELECT film_id, title FROM film;", con)
inventory     = pd.read_sql("SELECT inventory_id, film_id FROM inventory;", con)
rental        = pd.read_sql("SELECT rental_id, inventory_id FROM rental;", con)
payment       = pd.read_sql("SELECT rental_id, amount FROM payment;", con)

# 2) Join işlemleri
merged = (category
          .merge(film_category, on="category_id")
          .merge(film, on="film_id")
          .merge(inventory, on="film_id")
          .merge(rental, on="inventory_id")
          .merge(payment, on="rental_id"))

# 3) Film + kategori bazında kiralama sayısı ve gelir
film_stats = (merged
              .groupby(["category_id","name","film_id","title"], as_index=False)
              .agg(rental_count=("rental_id","count"),
                   total_revenue=("amount","sum")))

# 4) Her kategori içinde en yüksek kiralama sayısına sahip filmi seç
idx = film_stats.groupby("category_id")["rental_count"].idxmax()
pd_s10 = film_stats.loc[idx].sort_values("name").reset_index(drop=True)

pd_s10.head()


,category_id,name,film_id,title,rental_count,total_revenue
0,1,Action,748,RUGRATS SHAKESPEARE,30,70.70
1,2,Animation,489,JUGGLER HARDLY,32,96.68
2,3,Children,735,ROBBERS JOON,31,105.69
3,4,Classics,891,TIMBERLAND SKY,31,123.69
4,5,Comedy,1000,ZORRO ARK,31,214.69


# Soru 11: Tür ve Tarihe Göre Kiralama Sayısı ve Gelir

In [12]:
# Yalnız tablo çekme
category      = pd.read_sql("SELECT category_id, name FROM category;", con)
film_category = pd.read_sql("SELECT film_id, category_id FROM film_category;", con)
film          = pd.read_sql("SELECT film_id FROM film;", con)
inventory     = pd.read_sql("SELECT inventory_id, film_id FROM inventory;", con)
rental        = pd.read_sql("SELECT rental_id, inventory_id, rental_date FROM rental;", con)
payment       = pd.read_sql("SELECT rental_id, amount FROM payment;", con)

# Join
m = (category
     .merge(film_category, on="category_id")
     .merge(film, on="film_id")
     .merge(inventory, on="film_id")
     .merge(rental, on="inventory_id")
     .merge(payment, on="rental_id"))

m["rental_date"] = pd.to_datetime(m["rental_date"]).dt.date
pd_s11 = (m.groupby(["name","rental_date"], as_index=False)
            .agg(rental_count=("rental_id","count"),
                 total_revenue=("amount","sum"))
            .rename(columns={"name":"category_name"})
            .sort_values(["category_name","rental_date"]))
pd_s11.head()


,category_name,rental_date,rental_count,total_revenue
0,Action,2005-05-25,10,46.90
1,Action,2005-05-26,15,80.85
2,Action,2005-05-27,9,37.91
3,Action,2005-05-28,20,68.80
4,Action,2005-05-29,12,42.88


# Soru 12: Kiralanabilir Filmler İçin Türlerine Göre Her Filmin Kaç Kez Kiralandığı

In [13]:
# 1) Tablolar
category      = pd.read_sql("SELECT category_id, name FROM category;", con)
film_category = pd.read_sql("SELECT film_id, category_id FROM film_category;", con)
film          = pd.read_sql("SELECT film_id, title FROM film;", con)
inventory     = pd.read_sql("SELECT inventory_id, film_id FROM inventory;", con)
rental        = pd.read_sql("SELECT rental_id, inventory_id FROM rental;", con)

# 2) Join zinciri
m = (category
     .merge(film_category, on="category_id")
     .merge(film, on="film_id")
     .merge(inventory, on="film_id")
     .merge(rental, on="inventory_id"))

# 3) Agregasyon
pd_s12 = (m.groupby(["name","film_id","title"], as_index=False)
            .agg(rental_count=("rental_id","count"))
            .rename(columns={"name":"category_name"})
            .sort_values(["category_name","rental_count","title"],
                         ascending=[True, False, True])
            .reset_index(drop=True))

pd_s12.head()


,category_name,film_id,title,rental_count
0,Action,748,RUGRATS SHAKESPEARE,30
1,Action,869,SUSPECTS QUILLS,30
2,Action,395,HANDICAP BOONDOCK,28
3,Action,850,STORY SIDE,28
4,Action,911,TRIP NEWTON,28


# Soru 13: En çok rafta bekleyen filmler

In [14]:
from datetime import datetime, timezone
film      = pd.read_sql("SELECT film_id, title FROM film;", con)
inventory = pd.read_sql("SELECT inventory_id, film_id FROM inventory;", con)
rental    = pd.read_sql("SELECT rental_id, inventory_id, rental_date, return_date FROM rental;", con)
# 2) Tarihleri dönüştür
rental["rental_date"] = pd.to_datetime(rental["rental_date"])
rental["return_date"] = pd.to_datetime(rental["return_date"])
# 3) Her envanter için son kiralama kaydı
idx = rental.sort_values("rental_date").groupby("inventory_id")["rental_date"].idxmax()
last_pair = rental.loc[idx, ["inventory_id","return_date"]].merge(
    inventory[["inventory_id","film_id"]], on="inventory_id", how="left"
)
# 4) Rafta olanlar: son işlem iade edilmiş (return_date notna)
on_shelf = last_pair[last_pair["return_date"].notna()].copy()
# 5) Rafta bekleme günleri = bugün - return_date
today = pd.Timestamp(datetime.now(timezone.utc)).tz_localize(None)
on_shelf["shelf_days"] = (today - on_shelf["return_date"]).dt.total_seconds() / 86400.0
# 6) Film bazında ortalama bekleme ve kopya sayısı
pd_s13 = (on_shelf
          .groupby("film_id", as_index=False)
          .agg(avg_shelf_days=("shelf_days","mean"),
               copies_on_shelf=("inventory_id","count"))
          .merge(film, on="film_id", how="left")
          .sort_values("avg_shelf_days", ascending=False)
          .loc[:, ["film_id","title","avg_shelf_days","copies_on_shelf"]]
          .head(10)
          .reset_index(drop=True))
pd_s13.head(10)


,film_id,title,avg_shelf_days,copies_on_shelf
0,584,MIXED DOORS,7363.815202,2
1,501,KISSING DOLLS,7363.671071,5
2,611,MUSKETEERS WAIT,7361.612079,6
3,398,HANOVER GALAXY,7361.608379,4
4,23,ANACONDA CONFESSIONS,7361.500081,5
5,620,NEMO CAMPUS,7360.352150,5
6,399,HAPPINESS UNITED,7359.834861,1
7,448,IDAHO LOVE,7359.664011,5
8,371,GOSFORD DONNIE,7359.530331,3
9,899,TOWERS HURRICANE,7359.451358,3


# Soru 14: Geç, Erken ve Zamanında İade Edilen Kiralanmış Filmler

In [15]:
import numpy as np
# 1) Yalnız tablo çekme
film      = pd.read_sql("SELECT film_id, title, rental_duration FROM film;", con)
inventory = pd.read_sql("SELECT inventory_id, film_id FROM inventory;", con)
rental    = pd.read_sql("SELECT rental_id, inventory_id, rental_date, return_date FROM rental;", con)

# 2) Birleştir ve sadece iade edilenleri al
m = (rental.merge(inventory, on="inventory_id")
            .merge(film, on="film_id"))
m = m[m["return_date"].notna()].copy()

# 3) Tarihler ve vade
m["rental_date"] = pd.to_datetime(m["rental_date"]).dt.date
m["return_date"] = pd.to_datetime(m["return_date"]).dt.date
m["due_date"]    = m["rental_date"] + pd.to_timedelta(m["rental_duration"], unit="D")

# 4) İade durumu
conds = [
    m["return_date"] > m["due_date"],
    m["return_date"] < m["due_date"]
]
choices = ["Geç", "Erken"]
m["iade_durumu"] = np.select(conds, choices, default="Zamanında")

# 5) Film bazında özet
pd_s14 = (m.groupby(["film_id","title","iade_durumu"], as_index=False)
            .size()
            .pivot(index=["film_id","title"], columns="iade_durumu", values="size")
            .fillna(0)
            .astype(int)
            .reset_index()
            .sort_values(by=list(pd.Index(["Geç","Erken","Zamanında"]).intersection(_:=[])) or ["film_id"]))
pd_s14.head()

C:\Users\pc\AppData\Local\Temp\ipykernel_20420\2800619099.py:15: PerformanceWarning: Adding/subtracting object-dtype array to TimedeltaArray not vectorized.
  m["due_date"]    = m["rental_date"] + pd.to_timedelta(m["rental_duration"], unit="D")


iade_durumu,film_id,title,Erken,Geç,Zamanında
0,1,ACADEMY DINOSAUR,12,7,3
1,2,ACE GOLDFINGER,1,4,1
2,3,ADAPTATION HOLES,11,1,0
3,4,AFFAIR PREJUDICE,11,10,1
4,5,AFRICAN EGG,1,9,1


# Soru 15: Hangi müşteri en çok DVD kiralamış?

In [16]:
# 1) tabloları oku
rental   = pd.read_sql("SELECT rental_id, customer_id FROM rental;", con)
customer = pd.read_sql("SELECT customer_id, first_name, last_name FROM customer;", con)

# 2) müşteri bazında kiralama sayısı
cnt = (rental.groupby("customer_id", as_index=False)
             .agg(total_rentals=("rental_id","count")))

# 3) en yüksek değeri bul
mx = cnt["total_rentals"].max()
pd_s15 = (cnt[cnt["total_rentals"] == mx]
          .merge(customer, on="customer_id", how="left")
          .assign(customer_name=lambda d: d["first_name"]+" "+d["last_name"])
          .loc[:, ["customer_id","customer_name","total_rentals"]])

pd_s15

,customer_id,customer_name,total_rentals
0,148,ELEANOR HUNT,46


# Soru 16: En popüler film kategorisi nedir?

In [ ]:
# 1) tabloları oku
category      = pd.read_sql("SELECT category_id, name FROM category;", con)
film_category = pd.read_sql("SELECT film_id, category_id FROM film_category;", con)
inventory     = pd.read_sql("SELECT inventory_id, film_id FROM inventory;", con)
rental        = pd.read_sql("SELECT rental_id, inventory_id FROM rental;", con)

# 2) join zinciri
m = (category
     .merge(film_category, on="category_id")
     .merge(inventory, on="film_id")
     .merge(rental, on="inventory_id"))

# 3) kategori bazında kiralama sayısı
cat_counts = (m.groupby(["category_id","name"], as_index=False)
                .agg(rental_count=("rental_id","count")))

# 4) en yüksek
mx = cat_counts["rental_count"].max()
pd_s16 = cat_counts[cat_counts["rental_count"] == mx] \
          .rename(columns={"name":"category_name"}) \
          .reset_index(drop=True)<
pd_s16

,category_id,category_name,rental_count
0,15,Sports,1179


# Soru 17: Hangi çalışan en çok kiralama işlemi gerçekleştirmiş?

In [18]:
# 1) tabloları oku
staff  = pd.read_sql("SELECT staff_id, first_name, last_name FROM staff;", con)
rental = pd.read_sql("SELECT rental_id, staff_id FROM rental;", con)

# 2) çalışan bazında kiralama sayısı
cnt = (rental.groupby("staff_id", as_index=False)
             .agg(rental_count=("rental_id","count")))

# 3) maksimumu bul ve bilgileri ekle
mx = cnt["rental_count"].max()
pd_s17 = (cnt[cnt["rental_count"] == mx]
          .merge(staff, on="staff_id", how="left")
          .assign(staff_name=lambda d: d["first_name"]+" "+d["last_name"])
          .loc[:, ["staff_id","staff_name","rental_count"]])

pd_s17

,staff_id,staff_name,rental_count
0,1,Mike Hillyer,8040


# Soru 18: En çok geliri hangi film getirmiş?

In [19]:
film      = pd.read_sql("SELECT film_id, title FROM film;", con)
inventory = pd.read_sql("SELECT inventory_id, film_id FROM inventory;", con)
rental    = pd.read_sql("SELECT rental_id, inventory_id FROM rental;", con)
payment   = pd.read_sql("SELECT rental_id, amount FROM payment;", con)

m = (film.merge(inventory, on="film_id")
         .merge(rental, on="inventory_id")
         .merge(payment, on="rental_id"))

rev = (m.groupby(["film_id","title"], as_index=False)
        .agg(total_revenue=("amount","sum")))

mx = rev["total_revenue"].max()
pd_s18 = rev[rev["total_revenue"] == mx] \
           .sort_values(["film_id"]).reset_index(drop=True)

pd_s18

,film_id,title,total_revenue
0,879,TELEGRAPH VOYAGE,231.73


# Soru 19: Her müşteri için toplam harcama miktarını bulun

In [20]:
# 1) tablolar
customer = pd.read_sql("SELECT customer_id, first_name, last_name FROM customer;", con)
payment  = pd.read_sql("SELECT payment_id, customer_id, amount FROM payment;", con)

# 2) müşteri bazında toplam harcama
cust_spent = (payment.groupby("customer_id", as_index=False)
                       .agg(total_spent=("amount","sum")))

# 3) müşteri bilgileriyle birleştir
pd_s19 = (cust_spent.merge(customer, on="customer_id")
                   .assign(customer_name=lambda d: d["first_name"]+" "+d["last_name"])
                   .loc[:, ["customer_id","customer_name","total_spent"]]
                   .sort_values("total_spent", ascending=False)
                   .reset_index(drop=True))

pd_s19.head()

,customer_id,customer_name,total_spent
0,526,KARL SEAL,221.55
1,148,ELEANOR HUNT,216.54
2,144,CLARA SHAW,195.58
3,178,MARION SNYDER,194.61
4,137,RHONDA KENNEDY,194.61


# Soru 20: Her kategorideki toplam kiralama sayısını ve gelirleri bulun

In [21]:
category      = pd.read_sql("SELECT category_id, name FROM category;", con)
film_category = pd.read_sql("SELECT film_id, category_id FROM film_category;", con)
film          = pd.read_sql("SELECT film_id FROM film;", con)
inventory     = pd.read_sql("SELECT inventory_id, film_id FROM inventory;", con)
rental        = pd.read_sql("SELECT rental_id, inventory_id FROM rental;", con)
payment       = pd.read_sql("SELECT rental_id, amount FROM payment;", con)

m = (category
     .merge(film_category, on="category_id")
     .merge(film, on="film_id")
     .merge(inventory, on="film_id")
     .merge(rental, on="inventory_id")
     .merge(payment, on="rental_id"))

pd_s20 = (m.groupby("name", as_index=False)
           .agg(rental_count=("rental_id","count"),
                total_revenue=("amount","sum"))
           .rename(columns={"name":"category_name"})
           .sort_values(["total_revenue","rental_count"], ascending=False)
           .reset_index(drop=True))

pd_s20.head()


,category_name,rental_count,total_revenue
0,Sports,1179,5314.21
1,Sci-Fi,1101,4756.98
2,Animation,1166,4656.30
3,Drama,1060,4587.39
4,Comedy,941,4383.58


# Soru 21: En uzun süre kirada kalmış filmleri bulun

In [22]:
# 1) tablolar
film      = pd.read_sql("SELECT film_id, title FROM film;", con)
inventory = pd.read_sql("SELECT inventory_id, film_id FROM inventory;", con)
rental    = pd.read_sql("SELECT rental_id, inventory_id, rental_date, return_date FROM rental;", con)

# 2) iade edilen kiralamalar
r = rental[rental["return_date"].notna()].copy()
r["rental_date"] = pd.to_datetime(r["rental_date"])
r["return_date"] = pd.to_datetime(r["return_date"])
r["rental_days"] = (r["return_date"] - r["rental_date"]).dt.total_seconds() / 86400.0

# 3) film bazında maksimum kirada kalma süresi
m = (r.merge(inventory, on="inventory_id")
      .merge(film, on="film_id"))

pd_s21 = (m.groupby(["film_id","title"], as_index=False)
           .agg(max_rental_days=("rental_days","max"))
           .sort_values("max_rental_days", ascending=False)
           .reset_index(drop=True))

pd_s21.head()


,film_id,title,max_rental_days
0,416,HIGHBALL POTTER,9.249306
1,424,HOLOCAUST HIGHBALL,9.249306
2,172,CONEHEADS SMOOCHY,9.248611
3,906,TRAMP OTHERS,9.248611
4,561,MASK PEACH,9.248611


# Soru 22: En az kiralanan 5 film hangileridir?

In [23]:
# tablolar
film      = pd.read_sql("SELECT film_id, title FROM film;", con)
inventory = pd.read_sql("SELECT inventory_id, film_id FROM inventory;", con)
rental    = pd.read_sql("SELECT rental_id, inventory_id FROM rental;", con)

# join zinciri (LEFT JOIN)
m = (film
     .merge(inventory, on="film_id", how="left")
     .merge(rental, on="inventory_id", how="left"))

# film bazında kiralama sayısı
film_counts = (m.groupby(["film_id","title"], as_index=False)
                 .agg(rental_count=("rental_id","count"))
                 .sort_values(["rental_count","title"], ascending=[True, True])
                 .reset_index(drop=True))

# en az kiralanan 5 film
pd_s22 = film_counts.head(5)
pd_s22


,film_id,title,rental_count
0,14,ALICE FANTASIA,0
1,33,APOLLO TEEN,0
2,36,ARGONAUTS TOWN,0
3,38,ARK RIDGEMONT,0
4,41,ARSENIC INDEPENDENCE,0


# Soru 24: En fazla kazanç sağlayan 5 müşteriyi bulun

In [24]:
customer = pd.read_sql("SELECT customer_id, first_name, last_name FROM customer;", con)
payment  = pd.read_sql("SELECT customer_id, amount FROM payment;", con)

top5 = (payment.groupby("customer_id", as_index=False)
              .agg(total_spent=("amount","sum"))
              .merge(customer, on="customer_id", how="left")
              .assign(customer_name=lambda d: d["first_name"]+" "+d["last_name"])
              .loc[:, ["customer_id","customer_name","total_spent"]]
              .sort_values("total_spent", ascending=False)
              .head(5)
              .reset_index(drop=True))
top5


,customer_id,customer_name,total_spent
0,526,KARL SEAL,221.55
1,148,ELEANOR HUNT,216.54
2,144,CLARA SHAW,195.58
3,178,MARION SNYDER,194.61
4,137,RHONDA KENNEDY,194.61


# Soru 25: Her filmin ortalama kiralanma süresini bulun

In [25]:
# tablolar
film      = pd.read_sql("SELECT film_id, title FROM film;", con)
inventory = pd.read_sql("SELECT inventory_id, film_id FROM inventory;", con)
rental    = pd.read_sql("SELECT rental_id, inventory_id, rental_date, return_date FROM rental;", con)

# iade edilmiş kiralamalar
r = rental[rental["return_date"].notna()].copy()
r["rental_date"] = pd.to_datetime(r["rental_date"])
r["return_date"] = pd.to_datetime(r["return_date"])
r["rental_days"] = (r["return_date"] - r["rental_date"]).dt.total_seconds() / 86400.0

# film bazında ortalama kiralanma süresi
m = r.merge(inventory, on="inventory_id").merge(film, on="film_id")

pd_s25 = (m.groupby(["film_id","title"], as_index=False)
            .agg(avg_rental_days=("rental_days","mean"))
            .sort_values("avg_rental_days", ascending=False)
            .reset_index(drop=True))

pd_s25.head()


,film_id,title,avg_rental_days
0,323,FLIGHT LIES,7.270775
1,454,IMPACT ALADDIN,7.200617
2,5,AFRICAN EGG,7.106629
3,400,HARDLY ROBBERS,6.948438
4,546,MADRE GABLES,6.811806


# Soru 26: Her türde en popüler filmi bulun

In [26]:
# 1) tablolar
category      = pd.read_sql("SELECT category_id, name FROM category;", con)
film_category = pd.read_sql("SELECT film_id, category_id FROM film_category;", con)
film          = pd.read_sql("SELECT film_id, title FROM film;", con)
inventory     = pd.read_sql("SELECT inventory_id, film_id FROM inventory;", con)
rental        = pd.read_sql("SELECT rental_id, inventory_id FROM rental;", con)

# 2) join zinciri
m = (category
     .merge(film_category, on="category_id")
     .merge(film, on="film_id")
     .merge(inventory, on="film_id")
     .merge(rental, on="inventory_id"))

# 3) kategori+film bazında kiralama sayısı
film_counts = (m.groupby(["category_id","name","film_id","title"], as_index=False)
                .agg(rental_count=("rental_id","count")))

# 4) her kategori için en yüksek kiralama sayısını alan film(ler)
idx = film_counts.groupby("category_id")["rental_count"].idxmax()
pd_s26 = (film_counts.loc[idx]
          .rename(columns={"name":"category_name"})
          .sort_values(["category_name","title"])
          .reset_index(drop=True))

pd_s26  # columns: category_id, category_name, film_id, title, rental_count


,category_id,category_name,film_id,title,rental_count
0,1,Action,748,RUGRATS SHAKESPEARE,30
1,2,Animation,489,JUGGLER HARDLY,32
2,3,Children,735,ROBBERS JOON,31
3,4,Classics,891,TIMBERLAND SKY,31
4,5,Comedy,1000,ZORRO ARK,31
5,6,Documentary,973,WIFE TURN,31
6,7,Drama,418,HOBBIT ALIEN,31
7,8,Family,31,APACHE DIVINE,31
8,9,Foreign,738,ROCKETEER MOTHER,33
9,10,Games,331,FORWARD TEMPLE,32


# Soru 27: Her türde en fazla gelir sağlayan filmi bulun

In [27]:
category      = pd.read_sql("SELECT category_id, name FROM category;", con)
film_category = pd.read_sql("SELECT film_id, category_id FROM film_category;", con)
film          = pd.read_sql("SELECT film_id, title FROM film;", con)
inventory     = pd.read_sql("SELECT inventory_id, film_id FROM inventory;", con)
rental        = pd.read_sql("SELECT rental_id, inventory_id FROM rental;", con)
payment       = pd.read_sql("SELECT rental_id, amount FROM payment;", con)

m = (category
     .merge(film_category, on="category_id")
     .merge(film, on="film_id")
     .merge(inventory, on="film_id")
     .merge(rental, on="inventory_id")
     .merge(payment, on="rental_id"))

film_rev = (m.groupby(["category_id","name","film_id","title"], as_index=False)
              .agg(total_revenue=("amount","sum")))

idx = film_rev.groupby("category_id")["total_revenue"].idxmax()
pd_s27 = (film_rev.loc[idx]
          .rename(columns={"name":"category_name"})
          .sort_values(["category_name","title"])
          .reset_index(drop=True))

pd_s27  # category_id, category_name, film_id, title, total_revenue


,category_id,category_name,film_id,title,total_revenue
0,1,Action,327,FOOL MOCKINGBIRD,175.77
1,2,Animation,239,DOGMA FAMILY,178.70
2,3,Children,48,BACKLASH UNDEFEATED,158.81
3,4,Classics,843,STEEL SANTA,141.77
4,5,Comedy,1000,ZORRO ARK,214.69
5,6,Documentary,973,WIFE TURN,223.69
6,7,Drama,897,TORQUE BOUND,198.72
7,8,Family,715,RANGE MOONWALKER,179.73
8,9,Foreign,460,INNOCENT USUAL,191.74
9,10,Games,563,MASSACRE USUAL,179.70


# Soru 28: En çok DVD iade etmeyen müşteriyi bulun

In [28]:
rental   = pd.read_sql("SELECT customer_id, return_date FROM rental;", con)
customer = pd.read_sql("SELECT customer_id, first_name, last_name FROM customer;", con)

nr = (rental[rental["return_date"].isna()]
      .groupby("customer_id", as_index=False)
      .agg(not_returned=("return_date","size")))

if nr.empty:
    pd_s28 = pd.DataFrame(columns=["customer_id","customer_name","not_returned"])
else:
    mx = nr["not_returned"].max()
    pd_s28 = (nr[nr["not_returned"] == mx]
              .merge(customer, on="customer_id", how="left")
              .assign(customer_name=lambda d: d["first_name"]+" "+d["last_name"])
              .loc[:, ["customer_id","customer_name","not_returned"]]
              .sort_values("customer_id")
              .reset_index(drop=True))

pd_s28


,customer_id,customer_name,not_returned
0,75,TAMMY SANDERS,3


# Soru 29: En fazla kiralama yapan 5 çalışanı bulun

In [29]:
staff  = pd.read_sql("SELECT staff_id, first_name, last_name FROM staff;", con)
rental = pd.read_sql("SELECT rental_id, staff_id FROM rental;", con)

top5 = (rental.groupby("staff_id", as_index=False)
              .agg(rental_count=("rental_id","count"))
              .merge(staff, on="staff_id", how="left")
              .assign(staff_name=lambda d: d["first_name"]+" "+d["last_name"])
              .loc[:, ["staff_id","staff_name","rental_count"]]
              .sort_values("rental_count", ascending=False)
              .head(5)
              .reset_index(drop=True))
top5


,staff_id,staff_name,rental_count
0,1,Mike Hillyer,8040
1,2,Jon Stephens,8004


# Soru 30: En fazla kiralama yapan 5 müşteri hangi şubeden kiralama yapmış?

In [30]:


# tablolar
rental   = pd.read_sql("SELECT rental_id, customer_id FROM rental;", con)
customer = pd.read_sql("SELECT customer_id, first_name, last_name, store_id FROM customer;", con)
store    = pd.read_sql("SELECT store_id FROM store;", con)

# müşteri bazında kiralama sayısı
cust_counts = (rental.groupby("customer_id", as_index=False)
                       .agg(rental_count=("rental_id","count")))

# en çok kiralama yapan 5 müşteri
top5 = cust_counts.sort_values("rental_count", ascending=False).head(5)

# müşteri bilgileri ve şube ile birleştir
pd_s30 = (top5.merge(customer, on="customer_id", how="left")
               .merge(store, on="store_id", how="left")
               .assign(customer_name=lambda d: d["first_name"]+" "+d["last_name"])
               .loc[:, ["customer_id","customer_name","store_id","rental_count"]]
               .sort_values("rental_count", ascending=False)
               .reset_index(drop=True))

pd_s30


,customer_id,customer_name,store_id,rental_count
0,148,ELEANOR HUNT,1,46
1,526,KARL SEAL,2,45
2,236,MARCIA DEAN,1,42
3,144,CLARA SHAW,1,42
4,75,TAMMY SANDERS,2,41


# Soru 31: Her türde en az kiralanan filmi bulun

In [31]:
category      = pd.read_sql("SELECT category_id, name FROM category;", con)
film_category = pd.read_sql("SELECT film_id, category_id FROM film_category;", con)
film          = pd.read_sql("SELECT film_id, title FROM film;", con)
inventory     = pd.read_sql("SELECT inventory_id, film_id FROM inventory;", con)
rental        = pd.read_sql("SELECT rental_id, inventory_id FROM rental;", con)

# LEFT JOIN zinciri: kategori→film_category→film → (LEFT) inventory → (LEFT) rental
m = (category
     .merge(film_category, on="category_id")
     .merge(film, on="film_id")
     .merge(inventory, on="film_id", how="left")
     .merge(rental, on="inventory_id", how="left"))

# Kategori+film bazında kiralama sayısı (rental_id sayımı; NaN→0)
film_counts = (m.groupby(["category_id","name","film_id","title"], as_index=False)
                 .agg(rental_count=("rental_id","count"))
                 .rename(columns={"name":"category_name"}))

# Her kategori için minimumu seç (beraberlikleri dahil)
mins = film_counts.groupby("category_id")["rental_count"].transform("min")
pd_s31 = (film_counts[film_counts["rental_count"] == mins]
          .sort_values(["category_name","title"])
          .reset_index(drop=True))

pd_s31  # category_id, category_name, film_id, title, rental_count


,category_id,category_name,film_id,title,rental_count
0,1,Action,38,ARK RIDGEMONT,0
1,1,Action,318,FIREHOUSE VIETNAM,0
2,1,Action,802,SKY MIRACLE,0
3,2,Animation,36,ARGONAUTS TOWN,0
4,2,Animation,325,FLOATS GARDEN,0
5,3,Children,801,SISTER FREDDY,0
6,3,Children,955,WALLS ARTIST,0
7,4,Classics,14,ALICE FANTASIA,0
8,4,Classics,874,TADPOLE PARK,0
9,4,Classics,950,VOLUME HOUSE,0


# Soru 32: En çok kiralama yapan 5 müşteri hangi şehirde?

In [32]:
# tablolar
rental   = pd.read_sql("SELECT rental_id, customer_id FROM rental;", con)
customer = pd.read_sql("SELECT customer_id, first_name, last_name, address_id FROM customer;", con)
address  = pd.read_sql("SELECT address_id, city_id FROM address;", con)
city     = pd.read_sql("SELECT city_id, city FROM city;", con)

# müşteri bazında kiralama sayısı
cust_counts = (rental.groupby("customer_id", as_index=False)
                       .agg(rental_count=("rental_id","count")))

# en çok kiralama yapan 5 müşteri
top5 = cust_counts.sort_values("rental_count", ascending=False).head(5)

# müşteri + adres + şehir bilgisiyle birleştir
pd_s32 = (top5.merge(customer, on="customer_id", how="left")
               .merge(address, on="address_id", how="left")
               .merge(city, on="city_id", how="left")
               .assign(customer_name=lambda d: d["first_name"]+" "+d["last_name"])
               .loc[:, ["customer_id","customer_name","city","rental_count"]]
               .sort_values("rental_count", ascending=False)
               .reset_index(drop=True))

pd_s32


,customer_id,customer_name,city,rental_count
0,148,ELEANOR HUNT,Saint-Denis,46
1,526,KARL SEAL,Cape Coral,45
2,236,MARCIA DEAN,Tanza,42
3,144,CLARA SHAW,Molodetno,42
4,75,TAMMY SANDERS,Changhwa,41


# Soru 33: En çok kazanç sağlayan 5 müşteriyi hangi şehirde bulun?

In [33]:
payment  = pd.read_sql("SELECT customer_id, amount FROM payment;", con)
customer = pd.read_sql("SELECT customer_id, first_name, last_name, address_id FROM customer;", con)
address  = pd.read_sql("SELECT address_id, city_id FROM address;", con)
city     = pd.read_sql("SELECT city_id, city FROM city;", con)

# müşteri bazında toplam harcama
cust_rev = (payment.groupby("customer_id", as_index=False)
                    .agg(total_spent=("amount","sum"))
                    .sort_values("total_spent", ascending=False)
                    .head(5))

pd_s33 = (cust_rev
          .merge(customer, on="customer_id", how="left")
          .merge(address, on="address_id", how="left")
          .merge(city, on="city_id", how="left")
          .assign(customer_name=lambda d: d["first_name"]+" "+d["last_name"])
          .loc[:, ["customer_id","customer_name","city","total_spent"]]
          .sort_values("total_spent", ascending=False)
          .reset_index(drop=True))

pd_s33


,customer_id,customer_name,city,total_spent
0,526,KARL SEAL,Cape Coral,221.55
1,148,ELEANOR HUNT,Saint-Denis,216.54
2,144,CLARA SHAW,Molodetno,195.58
3,178,MARION SNYDER,Santa Brbara dOeste,194.61
4,137,RHONDA KENNEDY,Apeldoorn,194.61


# Soru 34: En çok kiralanan 5 filmi hangi şehirde bulun?

In [ ]:
payment  = pd.read_sql("SELECT customer_id, amount FROM payment;", con)
customer = pd.read_sql("SELECT customer_id, first_name, last_name, address_id FROM customer;", con)
address  = pd.read_sql("SELECT address_id, city_id FROM address;", con)
city     = pd.read_sql("SELECT city_id, city FROM city;", con)

# müşteri bazında toplam harcama
cust_rev = (payment.groupby("customer_id", as_index=False)
                    .agg(total_spent=("amount","sum"))
                    .sort_values("total_spent", ascending=False)
                    .head(5))

pd_s33 = (cust_rev
          .merge(customer, on="customer_id", how="left")
          .merge(address, on="address_id", how="left")
          .merge(city, on="city_id", how="left")
          .assign(customer_name=lambda d: d["first_name"]+" "+d["last_name"])
          .loc[:, ["customer_id","customer_name","city","total_spent"]]
          .sort_values("total_spent", ascending=False)
          .reset_index(drop=True))
pd_s33
<

,customer_id,customer_name,city,total_spent
0,526,KARL SEAL,Cape Coral,221.55
1,148,ELEANOR HUNT,Saint-Denis,216.54
2,144,CLARA SHAW,Molodetno,195.58
3,178,MARION SNYDER,Santa Brbara dOeste,194.61
4,137,RHONDA KENNEDY,Apeldoorn,194.61


# Soru 35: En az kiralanan 5 filmi hangi şehirde bulun?

In [35]:
film      = pd.read_sql("SELECT film_id, title FROM film;", con)
inventory = pd.read_sql("SELECT inventory_id, film_id, store_id FROM inventory;", con)
rental    = pd.read_sql("SELECT rental_id, inventory_id FROM rental;", con)
store     = pd.read_sql("SELECT store_id, address_id FROM store;", con)
address   = pd.read_sql("SELECT address_id, city_id FROM address;", con)
city      = pd.read_sql("SELECT city_id, city FROM city;", con)

# 1) Film bazında kiralama sayısı (LEFT JOIN mantığı)
m_counts = (film
            .merge(inventory[["inventory_id","film_id"]], on="film_id", how="left")
            .merge(rental[["rental_id","inventory_id"]], on="inventory_id", how="left"))

film_cnt = (m_counts.groupby(["film_id","title"], as_index=False)
                    .agg(rental_count=("rental_id","count"))
                    .sort_values(["rental_count","title"], ascending=[True, True]))

least5 = film_cnt.head(5)

# 2) Bu 5 filmin bulunduğu şehir(ler)
m_city = (inventory.merge(store, on="store_id", how="left")
                    .merge(address, on="address_id", how="left")
                    .merge(city, on="city_id", how="left"))

least5_cities = (least5.merge(m_city[["film_id","city"]], on="film_id", how="left")
                        .groupby(["film_id","title","rental_count"], as_index=False)
                        .agg(cities=("city", lambda x: ", ".join(sorted(set([c for c in x if pd.notna(c)]))))))

pd_s35 = least5_cities.sort_values(["rental_count","title"]).reset_index(drop=True)
pd_s35


,film_id,title,rental_count,cities
0,14,ALICE FANTASIA,0,
1,33,APOLLO TEEN,0,
2,36,ARGONAUTS TOWN,0,
3,38,ARK RIDGEMONT,0,
4,41,ARSENIC INDEPENDENCE,0,


# Soru 36: En çok kazanç sağlayan 5 filmi hangi şehirde bulun?

In [36]:
film      = pd.read_sql("SELECT film_id, title FROM film;", con)
inventory = pd.read_sql("SELECT inventory_id, film_id, store_id FROM inventory;", con)
rental    = pd.read_sql("SELECT rental_id, inventory_id FROM rental;", con)
payment   = pd.read_sql("SELECT rental_id, amount FROM payment;", con)
store     = pd.read_sql("SELECT store_id, address_id FROM store;", con)
address   = pd.read_sql("SELECT address_id, city_id FROM address;", con)
city      = pd.read_sql("SELECT city_id, city FROM city;", con)

# 1) Film geliri
rev = (film.merge(inventory, on="film_id")
            .merge(rental, on="inventory_id")
            .merge(payment, on="rental_id")
            .groupby(["film_id","title"], as_index=False)
            .agg(total_revenue=("amount","sum"))
            .sort_values(["total_revenue","title"], ascending=[False, True]))
top5 = rev.head(5)

# 2) Film→şehir eşlemesi
film_city = (inventory.merge(store, on="store_id")
                      .merge(address, on="address_id")
                      .merge(city, on="city_id")
                      .loc[:, ["film_id","city"]]
                      .dropna()
                      .drop_duplicates())  # DISTINCT

# 3) Şehirleri birleştir
pd_s36 = (top5.merge(film_city, on="film_id", how="left")
               .groupby(["film_id","title","total_revenue"], as_index=False)
               .agg(cities=("city", lambda x: ", ".join(sorted(set(x.dropna())))))
               .sort_values(["total_revenue","title"], ascending=[False, True])
               .reset_index(drop=True))
pd_s36


,film_id,title,total_revenue,cities
0,879,TELEGRAPH VOYAGE,231.73,"Lethbridge, Woodridge"
1,973,WIFE TURN,223.69,"Lethbridge, Woodridge"
2,1000,ZORRO ARK,214.69,"Lethbridge, Woodridge"
3,369,GOODFELLAS SALUTE,209.69,"Lethbridge, Woodridge"
4,764,SATURDAY LAMBS,204.72,"Lethbridge, Woodridge"


# Soru 37: En az kazanç sağlayan 5 filmi hangi şehirde bulun?

In [37]:
film      = pd.read_sql("SELECT film_id, title FROM film;", con)
inventory = pd.read_sql("SELECT inventory_id, film_id, store_id FROM inventory;", con)
rental    = pd.read_sql("SELECT rental_id, inventory_id FROM rental;", con)
payment   = pd.read_sql("SELECT rental_id, amount FROM payment;", con)
store     = pd.read_sql("SELECT store_id, address_id FROM store;", con)
address   = pd.read_sql("SELECT address_id, city_id FROM address;", con)
city      = pd.read_sql("SELECT city_id, city FROM city;", con)

# 1) Film geliri (LEFT JOIN zinciri, boşlara 0)
rev = (film
       .merge(inventory, on="film_id", how="left")
       .merge(rental,    on="inventory_id", how="left")
       .merge(payment,   on="rental_id", how="left")
       .groupby(["film_id","title"], as_index=False)
       .agg(total_revenue=("amount", lambda s: s.fillna(0).sum()))
       .sort_values(["total_revenue","title"], ascending=[True, True]))
least5 = rev.head(5)

# 2) Film→şehir eşlemesi (DISTINCT)
film_city = (inventory.merge(store, on="store_id")
                      .merge(address, on="address_id")
                      .merge(city, on="city_id")
                      .loc[:, ["film_id","city"]]
                      .dropna()
                      .drop_duplicates())

# 3) Şehirleri birleştir
pd_s37 = (least5.merge(film_city, on="film_id", how="left")
                 .groupby(["film_id","title","total_revenue"], as_index=False)
                 .agg(cities=("city", lambda x: ", ".join(sorted(set(x.dropna()))))))
pd_s37


,film_id,title,total_revenue,cities
0,261,DUFFEL APOCALYPSE,6.93,Lethbridge
1,335,FREEDOM CLEOPATRA,5.95,Lethbridge
2,635,OKLAHOMA JUMANJI,5.94,Woodridge
3,885,TEXAS WATCH,5.94,Lethbridge
4,996,YOUNG LANGUAGE,6.93,Lethbridge


# Soru 38: En fazla kiralama yapan müşteri hangi filmleri kiralamış?

In [38]:
rental   = pd.read_sql("SELECT rental_id, customer_id, inventory_id FROM rental;", con)
inventory= pd.read_sql("SELECT inventory_id, film_id FROM inventory;", con)
film     = pd.read_sql("SELECT film_id, title FROM film;", con)
customer = pd.read_sql("SELECT customer_id, first_name, last_name FROM customer;", con)

# 1) En çok kiralayan müşteri(ler)
cnt = (rental.groupby("customer_id", as_index=False)
             .agg(total_rentals=("rental_id","count")))
mx = cnt["total_rentals"].max()
topc = cnt[cnt["total_rentals"] == mx][["customer_id"]]

# 2) Bu müşterinin(lerin) kiraladığı filmler ve tekrar sayısı
m = (topc.merge(rental, on="customer_id")
          .merge(inventory, on="inventory_id")
          .merge(film, on="film_id"))

pd_s38 = (m.groupby(["customer_id","film_id","title"], as_index=False)
            .agg(times_rented=("rental_id","count"))
            .merge(customer, on="customer_id")
            .assign(customer_name=lambda d: d["first_name"]+" "+d["last_name"])
            .loc[:, ["customer_id","customer_name","film_id","title","times_rented"]]
            .sort_values(["customer_id","times_rented","title"],
                         ascending=[True, False, True])
            .reset_index(drop=True))

pd_s38


,customer_id,customer_name,film_id,title,times_rented
0,148,ELEANOR HUNT,4,AFFAIR PREJUDICE,1
1,148,ELEANOR HUNT,21,AMERICAN CIRCUS,1
2,148,ELEANOR HUNT,37,ARIZONA BANG,1
3,148,ELEANOR HUNT,39,ARMAGEDDON LOST,1
4,148,ELEANOR HUNT,75,BIRD INDEPENDENCE,1
5,148,ELEANOR HUNT,90,BOULEVARD MOB,1
6,148,ELEANOR HUNT,166,COLOR PHILADELPHIA,1
7,148,ELEANOR HUNT,167,COMA HEAD,1
8,148,ELEANOR HUNT,231,DINOSAUR SECRETARY,1
9,148,ELEANOR HUNT,275,EGYPT TENENBAUMS,1


# Soru 40: En çok kazanç sağlayan müşteri hangi filmleri kiralamış?

In [39]:
payment   = pd.read_sql("SELECT rental_id, customer_id, amount FROM payment;", con)
rental    = pd.read_sql("SELECT rental_id, customer_id, inventory_id FROM rental;", con)
inventory = pd.read_sql("SELECT inventory_id, film_id FROM inventory;", con)
film      = pd.read_sql("SELECT film_id, title FROM film;", con)
customer  = pd.read_sql("SELECT customer_id, first_name, last_name FROM customer;", con)

# 1) En çok harcayan müşteri(ler)
cust_rev = (payment.groupby("customer_id", as_index=False)
                    .agg(total_spent=("amount","sum")))
mxc = cust_rev["total_spent"].max()
topc = cust_rev[cust_rev["total_spent"] == mxc][["customer_id"]]

# 2) Bu müşterinin(lerin) kiraladığı filmler + adet + film bazında gelir
m = (topc.merge(rental, on="customer_id")
         .merge(payment[["rental_id","amount"]], on="rental_id")
         .merge(inventory, on="inventory_id")
         .merge(film, on="film_id"))

pd_s40 = (m.groupby(["customer_id","film_id","title"], as_index=False)
            .agg(times_rented=("rental_id","count"),
                 film_revenue=("amount","sum"))
            .merge(customer, on="customer_id")
            .assign(customer_name=lambda d: d["first_name"]+" "+d["last_name"])
            .loc[:, ["customer_id","customer_name","film_id","title","times_rented","film_revenue"]]
            .sort_values(["film_revenue","times_rented","title"],
                         ascending=[False, False, True])
            .reset_index(drop=True))
pd_s40


,customer_id,customer_name,film_id,title,times_rented,film_revenue
0,526,KARL SEAL,327,FOOL MOCKINGBIRD,1,10.99
1,526,KARL SEAL,846,STING PERSONAL,1,9.99
2,526,KARL SEAL,678,PICKUP DRIVING,1,8.99
3,526,KARL SEAL,884,TERMINATOR CLUB,1,8.99
4,526,KARL SEAL,412,HEAVYWEIGHTS BEAST,1,7.99
5,526,KARL SEAL,415,HIGH ENCINO,1,7.99
6,526,KARL SEAL,578,MILLION ACE,1,7.99
7,526,KARL SEAL,583,MISSION ZOOLANDER,1,7.99
8,526,KARL SEAL,966,WEDDING APOLLO,2,7.98
9,526,KARL SEAL,70,BIKINI BORROWERS,1,6.99


# Soru 41: En az kazanç sağlayan müşteri hangi filmleri kiralamış?

In [40]:
customer  = pd.read_sql("SELECT customer_id, first_name, last_name FROM customer;", con)
payment   = pd.read_sql("SELECT rental_id, customer_id, amount FROM payment;", con)
rental    = pd.read_sql("SELECT rental_id, customer_id, inventory_id FROM rental;", con)
inventory = pd.read_sql("SELECT inventory_id, film_id FROM inventory;", con)
film      = pd.read_sql("SELECT film_id, title FROM film;", con)

# 1) Müşteri toplam harcama (0'lar dahil)
cust_rev = (customer
            .merge(payment.groupby("customer_id", as_index=False)
                           .agg(total_spent=("amount","sum")),
                   on="customer_id", how="left")
            .assign(total_spent=lambda d: d["total_spent"].fillna(0.0))
            .assign(customer_name=lambda d: d["first_name"]+" "+d["last_name"])
            .loc[:, ["customer_id","customer_name","total_spent"]])

mn = cust_rev["total_spent"].min()
mincust = cust_rev[cust_rev["total_spent"] == mn][["customer_id","customer_name","total_spent"]]

# 2) Bu müşteri(ler)in kiraladığı filmler ve film bazında gelir
m = (mincust.merge(rental, on="customer_id", how="left")
             .merge(inventory, on="inventory_id", how="left")
             .merge(film, on="film_id", how="left")
             .merge(payment[["rental_id","amount"]], on="rental_id", how="left"))

pd_s41 = (m.groupby(["customer_id","customer_name","film_id","title"], as_index=False)
            .agg(times_rented=("rental_id","count"),
                 film_revenue=("amount", lambda s: s.fillna(0).sum()))
            .sort_values(["customer_id","film_revenue","times_rented","title"],
                         ascending=[True, True, True, True])
            .reset_index(drop=True))

pd_s41


,customer_id,customer_name,film_id,title,times_rented,film_revenue
0,248,CAROLINE BOWMAN,39,ARMAGEDDON LOST,1,0.99
1,248,CAROLINE BOWMAN,114,CAMELOT VACATION,1,0.99
2,248,CAROLINE BOWMAN,280,EMPIRE MALKOVICH,1,0.99
3,248,CAROLINE BOWMAN,452,ILLUSION AMELIE,1,0.99
4,248,CAROLINE BOWMAN,727,RESURRECTION SILVERADO,1,0.99
5,248,CAROLINE BOWMAN,885,TEXAS WATCH,1,0.99
6,248,CAROLINE BOWMAN,474,JADE BUNCH,1,2.99
7,248,CAROLINE BOWMAN,498,KILLER INNOCENT,1,2.99
8,248,CAROLINE BOWMAN,561,MASK PEACH,1,3.99
9,248,CAROLINE BOWMAN,369,GOODFELLAS SALUTE,1,4.99


# Soru 42: En az kazanç sağlayan müşteri hangi türde en fazla film kiralamış?

In [41]:
customer     = pd.read_sql("SELECT customer_id, first_name, last_name FROM customer;", con)
payment      = pd.read_sql("SELECT customer_id, amount FROM payment;", con)
rental       = pd.read_sql("SELECT rental_id, customer_id, inventory_id FROM rental;", con)
inventory    = pd.read_sql("SELECT inventory_id, film_id FROM inventory;", con)
film_category= pd.read_sql("SELECT film_id, category_id FROM film_category;", con)
category     = pd.read_sql("SELECT category_id, name FROM category;", con)

# 1) en az harcayan müşteri(ler)
cust_rev = (customer
            .merge(payment.groupby("customer_id", as_index=False)
                           .agg(total_spent=("amount","sum")),
                   on="customer_id", how="left")
            .assign(total_spent=lambda d: d["total_spent"].fillna(0.0))
            .assign(customer_name=lambda d: d["first_name"]+" "+d["last_name"])
            .loc[:, ["customer_id","customer_name","total_spent"]])
mn = cust_rev["total_spent"].min()
mincust = cust_rev[cust_rev["total_spent"] == mn][["customer_id","customer_name"]]

# 2) bu müşteri(ler)in tür bazında kiralama adedi
m = (mincust
     .merge(rental, on="customer_id", how="left")
     .merge(inventory, on="inventory_id", how="left")
     .merge(film_category, on="film_id", how="left")
     .merge(category, on="category_id", how="left"))

cust_cat = (m.groupby(["customer_id","customer_name","name"], as_index=False)
             .agg(rentals_in_cat=("rental_id","count"))
             .rename(columns={"name":"category_name"}))

# 3) müşteri başına maksimum tür adedini seç (beraberlikleri koru)
mx = cust_cat.groupby("customer_id")["rentals_in_cat"].transform("max")
pd_s42 = (cust_cat[cust_cat["rentals_in_cat"] == mx]
          .sort_values(["customer_id","category_name"])
          .reset_index(drop=True))

pd_s42  # columns: customer_id, customer_name, category_name, rentals_in_cat


,customer_id,customer_name,category_name,rentals_in_cat
0,248,CAROLINE BOWMAN,Sci-Fi,4


# Soru 43: En çok kiralanan film hangi çalışan tarafından kiralanmış?


In [42]:
film      = pd.read_sql("SELECT film_id, title FROM film;", con)
inventory = pd.read_sql("SELECT inventory_id, film_id FROM inventory;", con)
rental    = pd.read_sql("SELECT rental_id, inventory_id, staff_id FROM rental;", con)
staff     = pd.read_sql("SELECT staff_id, first_name, last_name FROM staff;", con)

# film bazında toplam kiralama
m_f = (film.merge(inventory, on="film_id")
           .merge(rental, on="inventory_id"))
film_cnt = (m_f.groupby(["film_id","title"], as_index=False)
              .agg(film_rentals=("rental_id","count")))

# en çok kiralanan film(ler)
mx = film_cnt["film_rentals"].max()
top_films = film_cnt[film_cnt["film_rentals"] == mx][["film_id","title","film_rentals"]]

# bu film(ler)i hangi çalışan kaç kez işlemiş?
sf = (top_films.merge(inventory, on="film_id")
               .merge(rental, on="inventory_id")
               .merge(staff, on="staff_id"))

staff_cnt = (sf.groupby(["film_id","title","film_rentals","staff_id","first_name","last_name"], as_index=False)
               .agg(rentals_by_staff=("rental_id","count"))
               .assign(staff_name=lambda d: d["first_name"]+" "+d["last_name"])
               .loc[:, ["film_id","title","film_rentals","staff_id","staff_name","rentals_by_staff"]])

# her film için en yüksek çalışan sayımı (beraberlikleri koru)
mx_staff = staff_cnt.groupby("film_id")["rentals_by_staff"].transform("max")
pd_s43 = (staff_cnt[staff_cnt["rentals_by_staff"] == mx_staff]
          .sort_values(["title","staff_name"])
          .reset_index(drop=True))

pd_s43  # columns: film_id, title, film_rentals, staff_id, staff_name, rentals_by_staff


,film_id,title,film_rentals,staff_id,staff_name,rentals_by_staff
0,103,BUCKET BROTHERHOOD,34,2,Jon Stephens,18


# Soru 44: En az kiralanan film hangi çalışan tarafından kiralanmış?

In [43]:
film      = pd.read_sql("SELECT film_id, title FROM film;", con)
inventory = pd.read_sql("SELECT inventory_id, film_id FROM inventory;", con)
rental    = pd.read_sql("SELECT rental_id, inventory_id, staff_id FROM rental;", con)
staff     = pd.read_sql("SELECT staff_id, first_name, last_name FROM staff;", con)

# 1) film bazında kiralama sayısı (yalnız kiralananlar)
m_f = (film.merge(inventory, on="film_id")
           .merge(rental, on="inventory_id"))
film_cnt = (m_f.groupby(["film_id","title"], as_index=False)
              .agg(film_rentals=("rental_id","count")))

mn = film_cnt["film_rentals"].min()
min_films = film_cnt[film_cnt["film_rentals"] == mn][["film_id","title","film_rentals"]]

# 2) bu film(ler)i işleyen çalışan(lar)
sf = (min_films.merge(inventory, on="film_id")
                .merge(rental, on="inventory_id")
                .merge(staff, on="staff_id"))

pd_s44 = (sf.groupby(["film_id","title","film_rentals","staff_id","first_name","last_name"], as_index=False)
            .agg(rentals_by_staff=("rental_id","count"))
            .assign(staff_name=lambda d: d["first_name"]+" "+d["last_name"])
            .loc[:, ["film_id","title","film_rentals","staff_id","staff_name","rentals_by_staff"]]
            .sort_values(["title","staff_name"])
            .reset_index(drop=True))

pd_s44


,film_id,title,film_rentals,staff_id,staff_name,rentals_by_staff
0,400,HARDLY ROBBERS,4,2,Jon Stephens,3
1,400,HARDLY ROBBERS,4,1,Mike Hillyer,1
2,584,MIXED DOORS,4,2,Jon Stephens,1
3,584,MIXED DOORS,4,1,Mike Hillyer,3
4,904,TRAIN BUNCH,4,2,Jon Stephens,4


# Soru 45: En çok kazanç sağlayan film hangi çalışan tarafından kiralanmış?

In [44]:
film      = pd.read_sql("SELECT film_id, title FROM film;", con)
inventory = pd.read_sql("SELECT inventory_id, film_id FROM inventory;", con)
rental    = pd.read_sql("SELECT rental_id, inventory_id, staff_id FROM rental;", con)
payment   = pd.read_sql("SELECT rental_id, amount FROM payment;", con)
staff     = pd.read_sql("SELECT staff_id, first_name, last_name FROM staff;", con)

# 1) film bazında toplam gelir
m = (film.merge(inventory, on="film_id")
          .merge(rental, on="inventory_id")
          .merge(payment, on="rental_id"))
film_rev = (m.groupby(["film_id","title"], as_index=False)
              .agg(total_revenue=("amount","sum")))

mx = film_rev["total_revenue"].max()
top_film = film_rev[film_rev["total_revenue"] == mx][["film_id","title","total_revenue"]]

# 2) bu film(ler) için çalışan bazında adet + gelir
sf = (top_film.merge(inventory, on="film_id")
               .merge(rental, on="inventory_id")
               .merge(payment, on="rental_id")
               .merge(staff, on="staff_id"))

pd_s45 = (sf.groupby(["film_id","title","total_revenue","staff_id","first_name","last_name"], as_index=False)
            .agg(rentals_by_staff=("rental_id","count"),
                 revenue_by_staff=("amount","sum"))
            .assign(staff_name=lambda d: d["first_name"]+" "+d["last_name"])
            .loc[:, ["film_id","title","total_revenue","staff_id","staff_name",
                     "rentals_by_staff","revenue_by_staff"]]
            .sort_values(["title","revenue_by_staff","rentals_by_staff","staff_name"],
                         ascending=[True, False, False, True])
            .reset_index(drop=True))

pd_s45


,film_id,title,total_revenue,staff_id,staff_name,rentals_by_staff,revenue_by_staff
0,879,TELEGRAPH VOYAGE,231.73,2,Jon Stephens,17,147.83
1,879,TELEGRAPH VOYAGE,231.73,1,Mike Hillyer,10,83.90


# Soru 46: En az kazanç sağlayan film hangi çalışan tarafından kiralanmış?

In [45]:
film      = pd.read_sql("SELECT film_id, title FROM film;", con)
inventory = pd.read_sql("SELECT inventory_id, film_id FROM inventory;", con)
rental    = pd.read_sql("SELECT rental_id, inventory_id, staff_id FROM rental;", con)
payment   = pd.read_sql("SELECT rental_id, amount FROM payment;", con)
staff     = pd.read_sql("SELECT staff_id, first_name, last_name FROM staff;", con)

# 1) film bazında toplam gelir (kiralanmış ve ödemesi olanlar)
m = (film.merge(inventory, on="film_id")
          .merge(rental, on="inventory_id")
          .merge(payment, on="rental_id"))
film_rev = (m.groupby(["film_id","title"], as_index=False)
              .agg(total_revenue=("amount","sum")))

mn = film_rev["total_revenue"].min()
min_film = film_rev[film_rev["total_revenue"] == mn][["film_id","title","total_revenue"]]

# 2) bu film(ler) için çalışan bazında adet + gelir
sf = (min_film.merge(inventory, on="film_id")
               .merge(rental, on="inventory_id")
               .merge(payment, on="rental_id")
               .merge(staff, on="staff_id"))

pd_s46 = (sf.groupby(["film_id","title","total_revenue","staff_id","first_name","last_name"], as_index=False)
            .agg(rentals_by_staff=("rental_id","count"),
                 revenue_by_staff=("amount","sum"))
            .assign(staff_name=lambda d: d["first_name"]+" "+d["last_name"])
            .loc[:, ["film_id","title","total_revenue","staff_id","staff_name",
                     "rentals_by_staff","revenue_by_staff"]]
            .sort_values(["title","revenue_by_staff","rentals_by_staff","staff_name"],
                         ascending=[True, True, True, True])
            .reset_index(drop=True))

pd_s46


,film_id,title,total_revenue,staff_id,staff_name,rentals_by_staff,revenue_by_staff
0,635,OKLAHOMA JUMANJI,5.94,2,Jon Stephens,2,1.98
1,635,OKLAHOMA JUMANJI,5.94,1,Mike Hillyer,4,3.96
2,885,TEXAS WATCH,5.94,2,Jon Stephens,3,2.97
3,885,TEXAS WATCH,5.94,1,Mike Hillyer,3,2.97


# Soru 47: En çok kiralanan film hangi mağazada kiralanmış?

In [46]:
film      = pd.read_sql("SELECT film_id, title FROM film;", con)
inventory = pd.read_sql("SELECT inventory_id, film_id, store_id FROM inventory;", con)
rental    = pd.read_sql("SELECT rental_id, inventory_id FROM rental;", con)

# 1) film bazında toplam kiralama
m_f = (film.merge(inventory, on="film_id")
           .merge(rental, on="inventory_id"))
film_cnt = (m_f.groupby(["film_id","title"], as_index=False)
              .agg(film_rentals=("rental_id","count")))

# 2) en çok kiralanan film(ler)
mx = film_cnt["film_rentals"].max()
top_films = film_cnt[film_cnt["film_rentals"] == mx][["film_id","title","film_rentals"]]

# 3) bu film(ler) için mağaza bazında kiralama adedi
sf = (top_films.merge(inventory, on="film_id")
               .merge(rental, on="inventory_id"))

store_cnt = (sf.groupby(["film_id","title","film_rentals","store_id"], as_index=False)
               .agg(rentals_in_store=("rental_id","count")))

# 4) film başına en yüksek mağaza adedini seç (beraberlikleri koru)
mx_store = store_cnt.groupby("film_id")["rentals_in_store"].transform("max")
pd_s47 = (store_cnt[store_cnt["rentals_in_store"] == mx_store]
          .sort_values(["title","store_id"])
          .reset_index(drop=True))

pd_s47  # columns: film_id, title, film_rentals, store_id, rentals_in_store


,film_id,title,film_rentals,store_id,rentals_in_store
0,103,BUCKET BROTHERHOOD,34,1,17
1,103,BUCKET BROTHERHOOD,34,2,17


# Soru 48: En az kiralanan film hangi mağazada kiralanmış?

In [47]:
film      = pd.read_sql("SELECT film_id, title FROM film;", con)
inventory = pd.read_sql("SELECT inventory_id, film_id, store_id FROM inventory;", con)
rental    = pd.read_sql("SELECT rental_id, inventory_id FROM rental;", con)

# 1) film bazında toplam kiralama
m_f = (film.merge(inventory, on="film_id")
           .merge(rental, on="inventory_id", how="left"))
film_cnt = (m_f.groupby(["film_id","title"], as_index=False)
              .agg(film_rentals=("rental_id","count")))

# 2) en az kiralanan film(ler)
mn = film_cnt["film_rentals"].min()
min_films = film_cnt[film_cnt["film_rentals"] == mn][["film_id","title","film_rentals"]]

# 3) bu film(ler) için mağaza bazında kiralama adedi
sf = (min_films.merge(inventory, on="film_id")
                 .merge(rental, on="inventory_id", how="left"))

store_cnt = (sf.groupby(["film_id","title","film_rentals","store_id"], as_index=False)
               .agg(rentals_in_store=("rental_id","count")))

# 4) her film için minimum mağaza adedini seç (beraberlikleri koru)
mn_store = store_cnt.groupby("film_id")["rentals_in_store"].transform("min")
pd_s48 = (store_cnt[store_cnt["rentals_in_store"] == mn_store]
          .sort_values(["title","store_id"])
          .reset_index(drop=True))

pd_s48   # columns: film_id, title, film_rentals, store_id, rentals_in_store


,film_id,title,film_rentals,store_id,rentals_in_store
0,400,HARDLY ROBBERS,4,1,4
1,584,MIXED DOORS,4,1,4
2,904,TRAIN BUNCH,4,1,4


# Soru 49: En çok kazanç sağlayan film hangi mağazada kiralanmış?

In [48]:
film      = pd.read_sql("SELECT film_id, title FROM film;", con)
inventory = pd.read_sql("SELECT inventory_id, film_id, store_id FROM inventory;", con)
rental    = pd.read_sql("SELECT rental_id, inventory_id FROM rental;", con)
payment   = pd.read_sql("SELECT rental_id, amount FROM payment;", con)

# 1) film bazında toplam gelir
m = (film.merge(inventory, on="film_id")
          .merge(rental, on="inventory_id")
          .merge(payment, on="rental_id"))

film_rev = (m.groupby(["film_id","title"], as_index=False)
              .agg(total_revenue=("amount","sum")))

mx = film_rev["total_revenue"].max()
top_film = film_rev[film_rev["total_revenue"] == mx][["film_id","title","total_revenue"]]

# 2) bu film(ler) için mağaza bazında gelir ve adet
sf = (top_film.merge(inventory, on="film_id")
               .merge(rental, on="inventory_id")
               .merge(payment, on="rental_id"))

store_rev = (sf.groupby(["film_id","title","total_revenue","store_id"], as_index=False)
               .agg(revenue_in_store=("amount","sum"),
                    rentals_in_store=("rental_id","count")))

# 3) film başına en yüksek mağaza gelirini seç (beraberlikleri koru)
mx_store = store_rev.groupby("film_id")["revenue_in_store"].transform("max")
pd_s49 = (store_rev[store_rev["revenue_in_store"] == mx_store]
          .sort_values(["title","store_id"])
          .reset_index(drop=True))

pd_s49  # film_id, title, total_revenue, store_id, revenue_in_store, rentals_in_store


,film_id,title,total_revenue,store_id,revenue_in_store,rentals_in_store
0,879,TELEGRAPH VOYAGE,231.73,1,132.85,15


# Soru 50: En az kazanç sağlayan film hangi mağazada kiralanmış?

In [49]:
film      = pd.read_sql("SELECT film_id, title FROM film;", con)
inventory = pd.read_sql("SELECT inventory_id, film_id, store_id FROM inventory;", con)
rental    = pd.read_sql("SELECT rental_id, inventory_id FROM rental;", con)
payment   = pd.read_sql("SELECT rental_id, amount FROM payment;", con)

# 1) Film bazında toplam gelir (yoksa 0)
m = (film.merge(inventory, on="film_id", how="left")
          .merge(rental, on="inventory_id", how="left")
          .merge(payment, on="rental_id", how="left"))

film_rev = (m.groupby(["film_id","title"], as_index=False)
              .agg(total_revenue=("amount", lambda s: s.fillna(0).sum())))

mn = film_rev["total_revenue"].min()
min_film = film_rev[film_rev["total_revenue"] == mn][["film_id","title","total_revenue"]]

# 2) Bu film(ler) için mağaza bazında gelir ve adet
sf = (min_film.merge(inventory, on="film_id", how="left")
               .merge(rental, on="inventory_id", how="left")
               .merge(payment, on="rental_id", how="left"))

store_rev = (sf.groupby(["film_id","title","total_revenue","store_id"], as_index=False)
               .agg(revenue_in_store=("amount", lambda s: s.fillna(0).sum()),
                    rentals_in_store=("rental_id","count")))

# 3) Film başına en yüksek mağaza gelirini seç (beraberlikleri korur)
mx_store = store_rev.groupby("film_id")["revenue_in_store"].transform("max")
pd_s50 = (store_rev[store_rev["revenue_in_store"] == mx_store]
          .sort_values(["title","store_id"])
          .reset_index(drop=True))

pd_s50  # film_id, title, total_revenue, store_id, revenue_in_store, rentals_in_store


,film_id,title,total_revenue,store_id,revenue_in_store,rentals_in_store
0,635,OKLAHOMA JUMANJI,5.94,2,5.94,6
1,885,TEXAS WATCH,5.94,1,5.94,6


# Soru 51: Müşterilerin kiraladıkları filmlerin toplam kiralama süresi ne kadar?

In [50]:
# tablolar
rental   = pd.read_sql("SELECT rental_id, customer_id, rental_date, return_date FROM rental;", con)
customer = pd.read_sql("SELECT customer_id, first_name, last_name FROM customer;", con)

# sadece iade edilmiş kiralamalar
r = rental[rental["return_date"].notna()].copy()
r["rental_date"] = pd.to_datetime(r["rental_date"])
r["return_date"] = pd.to_datetime(r["return_date"])

# kiralama süresi (gün)
r["days"] = (r["return_date"] - r["rental_date"]).dt.total_seconds() / 86400.0

# müşteri bazında toplam gün
pd_s51 = (r.groupby("customer_id", as_index=False)
            .agg(total_days=("days","sum"))
            .merge(customer, on="customer_id", how="left")
            .assign(customer_name=lambda d: d["first_name"]+" "+d["last_name"])
            .loc[:, ["customer_id","customer_name","total_days"]]
            .sort_values("total_days", ascending=False)
            .reset_index(drop=True))
pd_s51


,customer_id,customer_name,total_days
0,526,KARL SEAL,264.151389
1,148,ELEANOR HUNT,243.095139
2,144,CLARA SHAW,235.040278
3,137,RHONDA KENNEDY,229.187500
4,295,DAISY BATES,221.306250
...,...,...,...
594,248,CAROLINE BOWMAN,72.142361
595,395,JOHNNY TURPIN,69.217361
596,97,ANNIE RUSSELL,64.425694
597,318,BRIAN WYMAN,59.231944


# Soru 52: En çok kiralanan türdeki filmler hangileri?

In [51]:
category      = pd.read_sql("SELECT category_id, name FROM category;", con)
film_category = pd.read_sql("SELECT film_id, category_id FROM film_category;", con)
film          = pd.read_sql("SELECT film_id, title FROM film;", con)
inventory     = pd.read_sql("SELECT inventory_id, film_id FROM inventory;", con)
rental        = pd.read_sql("SELECT rental_id, inventory_id FROM rental;", con)

# 1) tür bazında toplam kiralama
m_cat = (category.merge(film_category, on="category_id")
                 .merge(film, on="film_id")
                 .merge(inventory, on="film_id")
                 .merge(rental, on="inventory_id"))

cat_cnt = (m_cat.groupby(["category_id","name"], as_index=False)
                 .agg(rental_count=("rental_id","count"))
                 .rename(columns={"name":"category_name"}))

# 2) en çok kiralanan tür(ler)
mx = cat_cnt["rental_count"].max()
top_cat = cat_cnt[cat_cnt["rental_count"] == mx][["category_id","category_name"]]

# 3) bu türdeki filmler ve kiralanma sayıları
m_top = (top_cat.merge(film_category, on="category_id")
                  .merge(film, on="film_id")
                  .merge(inventory, on="film_id")
                  .merge(rental, on="inventory_id"))

pd_s52 = (m_top.groupby(["film_id","title","category_name"], as_index=False)
               .agg(film_rentals=("rental_id","count"))
               .sort_values(["film_rentals","title"], ascending=[False, True])
               .reset_index(drop=True))

pd_s52


,film_id,title,category_name,film_rentals
0,361,GLEAMING JAWBREAKER,Sports,29
1,875,TALENTED HOMICIDE,Sports,29
2,745,ROSES TREASURE,Sports,28
3,764,SATURDAY LAMBS,Sports,28
4,135,CHANCE RESURRECTION,Sports,27
...,...,...,...,...
68,779,SENSE GREEK,Sports,7
69,836,SQUAD FISH,Sports,7
70,102,BUBBLE GROSSE,Sports,6
71,372,GRACELAND DYNAMITE,Sports,6


# Soru 53: En az kiralanan türdeki filmler hangileri?

In [52]:
category      = pd.read_sql("SELECT category_id, name FROM category;", con)
film_category = pd.read_sql("SELECT film_id, category_id FROM film_category;", con)
film          = pd.read_sql("SELECT film_id, title FROM film;", con)
inventory     = pd.read_sql("SELECT inventory_id, film_id FROM inventory;", con)
rental        = pd.read_sql("SELECT rental_id, inventory_id FROM rental;", con)

# 1) Tür bazında toplam kiralama (LEFT mantığı: hiç kiralanmayanlar dahil)
m_cat = (category.merge(film_category, on="category_id")
                 .merge(film, on="film_id")
                 .merge(inventory, on="film_id")
                 .merge(rental, on="inventory_id", how="left"))

cat_cnt = (m_cat.groupby(["category_id","name"], as_index=False)
                 .agg(rental_count=("rental_id","count"))
                 .rename(columns={"name":"category_name"}))

# 2) En az kiralanan tür(ler)
mn = cat_cnt["rental_count"].min()
min_cat = cat_cnt[cat_cnt["rental_count"] == mn][["category_id","category_name"]]

# 3) Bu türdeki filmler ve kiralanma sayıları
m_min = (min_cat.merge(film_category, on="category_id")
                  .merge(film, on="film_id")
                  .merge(inventory, on="film_id")
                  .merge(rental, on="inventory_id", how="left"))

pd_s53 = (m_min.groupby(["film_id","title","category_name"], as_index=False)
               .agg(film_rentals=("rental_id","count"))
               .sort_values(["film_rentals","title"], ascending=[True, True])
               .reset_index(drop=True))

pd_s53


,film_id,title,category_name,film_rentals
0,884,TERMINATOR CLUB,Music,6
1,997,YOUTH KICK,Music,6
2,161,CLUE GRAIL,Music,7
3,278,ELF MURDER,Music,7
4,411,HEAVENLY GUN,Music,7
5,384,GROSSE WONDERFUL,Music,8
6,718,REBEL AIRPORT,Music,8
7,752,RUNNER MADIGAN,Music,8
8,76,BIRDCAGE CASPER,Music,9
9,454,IMPACT ALADDIN,Music,9


# Soru 54: Her müşteri için toplam ödeme miktarını bulun.

In [53]:
customer = pd.read_sql("SELECT customer_id, first_name, last_name FROM customer;", con)
payment  = pd.read_sql("SELECT customer_id, amount FROM payment;", con)

pd_s54 = (payment.groupby("customer_id", as_index=False)
                 .agg(total_paid=("amount","sum"))
                 .merge(customer, on="customer_id", how="left")
                 .assign(customer_name=lambda d: d["first_name"]+" "+d["last_name"])
                 .loc[:, ["customer_id","customer_name","total_paid"]]
                 .sort_values("total_paid", ascending=False)
                 .reset_index(drop=True))
pd_s54


,customer_id,customer_name,total_paid
0,526,KARL SEAL,221.55
1,148,ELEANOR HUNT,216.54
2,144,CLARA SHAW,195.58
3,178,MARION SNYDER,194.61
4,137,RHONDA KENNEDY,194.61
...,...,...,...
594,97,ANNIE RUSSELL,58.82
595,395,JOHNNY TURPIN,57.81
596,318,BRIAN WYMAN,52.88
597,281,LEONA OBRIEN,50.86


# Soru 55: Hangi filmler en uzun süre kiralanmış?

In [54]:
film      = pd.read_sql("SELECT film_id, title FROM film;", con)
inventory = pd.read_sql("SELECT inventory_id, film_id FROM inventory;", con)
rental    = pd.read_sql("SELECT rental_id, inventory_id, rental_date, return_date FROM rental;", con)

# yalnızca iade edilmiş kiralamalar
r = rental[rental["return_date"].notna()].copy()
r["rental_date"] = pd.to_datetime(r["rental_date"])
r["return_date"] = pd.to_datetime(r["return_date"])
r["days"] = (r["return_date"] - r["rental_date"]).dt.total_seconds() / 86400.0

# film bazında en uzun süre
m = r.merge(inventory, on="inventory_id").merge(film, on="film_id")

pd_s55 = (m.groupby(["film_id","title"], as_index=False)
            .agg(max_days=("days","max"))
            .sort_values(["max_days","title"], ascending=[False, True])
            .reset_index(drop=True))

pd_s55


,film_id,title,max_days
0,416,HIGHBALL POTTER,9.249306
1,424,HOLOCAUST HIGHBALL,9.249306
2,172,CONEHEADS SMOOCHY,9.248611
3,561,MASK PEACH,9.248611
4,653,PANIC CLUB,9.248611
...,...,...,...
953,836,SQUAD FISH,5.908333
954,180,CONSPIRACY SPIRIT,5.809722
955,362,GLORY TRACY,5.120833
956,903,TRAFFIC HOBBIT,4.977778


# Soru 56: Hangi filmler en kısa süre kiralanmış?

In [55]:
film      = pd.read_sql("SELECT film_id, title FROM film;", con)
inventory = pd.read_sql("SELECT inventory_id, film_id FROM inventory;", con)
rental    = pd.read_sql("SELECT rental_id, inventory_id, rental_date, return_date FROM rental;", con)

r = rental[rental["return_date"].notna()].copy()
r["rental_date"] = pd.to_datetime(r["rental_date"])
r["return_date"] = pd.to_datetime(r["return_date"])
r["days"] = (r["return_date"] - r["rental_date"]).dt.total_seconds() / 86400.0

m = r.merge(inventory, on="inventory_id").merge(film, on="film_id")
pd_s56 = (m.groupby(["film_id","title"], as_index=False)
            .agg(min_days=("days","min"))
            .sort_values(["min_days","title"], ascending=[True, True])
            .reset_index(drop=True))
pd_s56


,film_id,title,min_days
0,200,CURTAIN VIDEOTAPE,0.750000
1,233,DISCIPLE MOTHER,0.750000
2,734,ROAD ROXANNE,0.750694
3,313,FIDELITY DEVIL,0.751389
4,410,HEAVEN FREEDOM,0.751389
...,...,...,...
953,584,MIXED DOORS,3.240972
954,268,EARLY HOME,3.807639
955,654,PANKY SUBMARINE,3.827083
956,952,WAGON JAWS,3.859028


# Soru 57: Müşterilerin kiraladığı filmler için ortalama ödeme miktarını bulun.

In [56]:
customer = pd.read_sql("SELECT customer_id, first_name, last_name FROM customer;", con)
payment  = pd.read_sql("SELECT customer_id, amount FROM payment;", con)

pd_s57 = (payment.groupby("customer_id", as_index=False)
                 .agg(avg_payment=("amount","mean"))
                 .merge(customer, on="customer_id", how="left")
                 .assign(customer_name=lambda d: d["first_name"]+" "+d["last_name"])
                 .loc[:, ["customer_id","customer_name","avg_payment"]]
                 .sort_values("avg_payment", ascending=False)
                 .reset_index(drop=True))
pd_s57


,customer_id,customer_name,avg_payment
0,187,BRITTANY RILEY,5.704286
1,433,DON BONE,5.350000
2,321,KEVIN SCHULER,5.308182
3,542,LONNIE TIRADO,5.267778
4,311,PAUL TROUT,5.250870
...,...,...,...
594,115,WENDY HARRISON,3.056667
595,64,JUDITH COX,3.050606
596,381,BOBBY BOUDREAU,3.047143
597,395,JOHNNY TURPIN,3.042632


# Soru 58: En çok kiralanan filmlerin ortalama kiralama süresi nedir?

In [57]:
film      = pd.read_sql("SELECT film_id, title FROM film;", con)
inventory = pd.read_sql("SELECT inventory_id, film_id FROM inventory;", con)
rental    = pd.read_sql("SELECT rental_id, inventory_id, rental_date, return_date FROM rental;", con)

# 1) film bazında kiralama sayısı
m_cnt = (film.merge(inventory, on="film_id")
              .merge(rental, on="inventory_id"))
film_cnt = (m_cnt.groupby(["film_id","title"], as_index=False)
              .agg(film_rentals=("rental_id","count")))

# 2) en çok kiralanan film(ler)
mx = film_cnt["film_rentals"].max()
top_films = film_cnt[film_cnt["film_rentals"] == mx][["film_id","title","film_rentals"]]

# 3) sadece iade edilmiş kiralamalar için süre hesabı
r = rental[rental["return_date"].notna()].copy()
r["rental_date"] = pd.to_datetime(r["rental_date"])
r["return_date"] = pd.to_datetime(r["return_date"])
r["days"] = (r["return_date"] - r["rental_date"]).dt.total_seconds() / 86400.0

# 4) top film(ler) için ortalama kiralama süresi
pd_s58 = (top_films.merge(inventory, on="film_id")
                   .merge(r, on="inventory_id")
                   .groupby(["film_id","title","film_rentals"], as_index=False)
                   .agg(avg_rental_days=("days","mean"))
                   .sort_values("title")
                   .reset_index(drop=True))
pd_s58


,film_id,title,film_rentals,avg_rental_days
0,103,BUCKET BROTHERHOOD,34,4.948448


# Soru 59: En az kiralanan filmlerin ortalama kiralama süresi nedir?

In [58]:
film      = pd.read_sql("SELECT film_id, title FROM film;", con)
inventory = pd.read_sql("SELECT inventory_id, film_id FROM inventory;", con)
rental    = pd.read_sql("SELECT rental_id, inventory_id, rental_date, return_date FROM rental;", con)

# 1) film bazında toplam kiralama
m_cnt = (film.merge(inventory, on="film_id")
              .merge(rental, on="inventory_id", how="left"))
film_cnt = (m_cnt.groupby(["film_id","title"], as_index=False)
              .agg(film_rentals=("rental_id","count")))

# 2) en az kiralanan film(ler)
mn = film_cnt["film_rentals"].min()
min_films = film_cnt[film_cnt["film_rentals"] == mn][["film_id","title","film_rentals"]]

# 3) iade edilmiş kiralamalar için süre hesabı
r = rental[rental["return_date"].notna()].copy()
r["rental_date"] = pd.to_datetime(r["rental_date"])
r["return_date"] = pd.to_datetime(r["return_date"])
r["days"] = (r["return_date"] - r["rental_date"]).dt.total_seconds() / 86400.0

# 4) bu filmler için ortalama kiralama süresi
pd_s59 = (min_films.merge(inventory, on="film_id")
                     .merge(r, on="inventory_id")
                     .groupby(["film_id","title","film_rentals"], as_index=False)
                     .agg(avg_rental_days=("days","mean"))
                     .sort_values("title")
                     .reset_index(drop=True))
pd_s59


,film_id,title,film_rentals,avg_rental_days
0,400,HARDLY ROBBERS,4,6.948437
1,584,MIXED DOORS,4,5.953299
2,904,TRAIN BUNCH,4,3.349306


# Soru 60: En çok kazanç sağlayan müşterilerin ortalama ödeme miktarı nedir?

In [59]:
payment  = pd.read_sql("SELECT customer_id, amount FROM payment;", con)
customer = pd.read_sql("SELECT customer_id, first_name, last_name FROM customer;", con)

# 1) müşteri toplam harcama
cust_rev = (payment.groupby("customer_id", as_index=False)
                    .agg(total_spent=("amount","sum")))

# 2) en çok harcayan müşteri(ler)
mx = cust_rev["total_spent"].max()
topc = cust_rev[cust_rev["total_spent"] == mx][["customer_id"]]

# 3) bu müşteri(ler) için ortalama ödeme
avg_pay = (payment.merge(topc, on="customer_id")
                  .groupby("customer_id", as_index=False)
                  .agg(avg_payment=("amount","mean"))
                  .merge(customer, on="customer_id")
                  .assign(customer_name=lambda d: d["first_name"]+" "+d["last_name"])
                  .loc[:, ["customer_id","customer_name","avg_payment"]]
                  .sort_values("avg_payment", ascending=False)
                  .reset_index(drop=True))

avg_pay


,customer_id,customer_name,avg_payment
0,526,KARL SEAL,4.923333


# Soru 61: En az kazanç sağlayan müşterilerin ortalama ödeme miktarı nedir?

In [60]:
payment  = pd.read_sql("SELECT customer_id, amount FROM payment;", con)
customer = pd.read_sql("SELECT customer_id, first_name, last_name FROM customer;", con)

# 1) müşteri toplam harcama
cust_rev = (payment.groupby("customer_id", as_index=False)
                    .agg(total_spent=("amount","sum")))

# 2) en az harcayan müşteri(ler)
mn = cust_rev["total_spent"].min()
minc = cust_rev[cust_rev["total_spent"] == mn][["customer_id"]]

# 3) ortalama ödeme (işlem başına); ödeme kaydı yoksa NaN
pd_s61 = (minc.merge(payment, on="customer_id", how="left")
               .groupby("customer_id", as_index=False)
               .agg(avg_payment=("amount","mean"))
               .merge(customer, on="customer_id", how="left")
               .assign(customer_name=lambda d: d["first_name"]+" "+d["last_name"])
               .loc[:, ["customer_id","customer_name","avg_payment"]]
               .sort_values("avg_payment", ascending=True)
               .reset_index(drop=True))
pd_s61


,customer_id,customer_name,avg_payment
0,248,CAROLINE BOWMAN,3.39


# Soru 62: Mağazalardaki toplam kiralama süresini bulun.

In [61]:
store     = pd.read_sql("SELECT store_id FROM store;", con)
inventory = pd.read_sql("SELECT inventory_id, store_id FROM inventory;", con)
rental    = pd.read_sql("SELECT inventory_id, rental_date, return_date FROM rental;", con)

# yalnızca iade edilmişler
r = rental[rental["return_date"].notna()].copy()
r["rental_date"] = pd.to_datetime(r["rental_date"])
r["return_date"] = pd.to_datetime(r["return_date"])
r["days"] = (r["return_date"] - r["rental_date"]).dt.total_seconds() / 86400.0

pd_s62 = (inventory.merge(r, on="inventory_id")
                  .groupby("store_id", as_index=False)
                  .agg(total_days=("days","sum"))
                  .sort_values("store_id")
                  .reset_index(drop=True))
pd_s62


,store_id,total_days
0,1,39439.421528
1,2,40267.343056


# Soru 63: En uzun süre kiralanan film hangi mağazada kiralanmış?

In [62]:
film      = pd.read_sql("SELECT film_id, title FROM film;", con)
inventory = pd.read_sql("SELECT inventory_id, film_id, store_id FROM inventory;", con)
rental    = pd.read_sql("SELECT rental_id, inventory_id, rental_date, return_date FROM rental;", con)

# 1) yalnız iade edilmiş kayıtlar
r = rental[rental["return_date"].notna()].copy()
r["rental_date"] = pd.to_datetime(r["rental_date"])
r["return_date"] = pd.to_datetime(r["return_date"])
r["days"] = (r["return_date"] - r["rental_date"]).dt.total_seconds() / 86400.0

# 2) film bazında max süre
m_film = r.merge(inventory, on="inventory_id").merge(film, on="film_id")
film_dur = (m_film.groupby(["film_id","title"], as_index=False)
                   .agg(max_days=("days","max")))

mx = film_dur["max_days"].max()
longest = film_dur[film_dur["max_days"] == mx][["film_id","title","max_days"]]

# 3) mağaza bilgisi
pd_s63 = (longest.merge(inventory, on="film_id")
                   .merge(r, on="inventory_id")
                   .loc[:, ["film_id","title","max_days","store_id"]]
                   .drop_duplicates()
                   .sort_values("store_id")
                   .reset_index(drop=True))

pd_s63


,film_id,title,max_days,store_id
0,416,HIGHBALL POTTER,9.249306,1
1,424,HOLOCAUST HIGHBALL,9.249306,1
2,416,HIGHBALL POTTER,9.249306,2
3,424,HOLOCAUST HIGHBALL,9.249306,2


# Soru 64: En kısa süre kiralanan film hangi mağazada kiralanmış?

In [63]:
film      = pd.read_sql("SELECT film_id, title FROM film;", con)
inventory = pd.read_sql("SELECT inventory_id, film_id, store_id FROM inventory;", con)
rental    = pd.read_sql("SELECT rental_id, inventory_id, rental_date, return_date FROM rental;", con)

# 1) yalnız iade edilmiş kayıtlar
r = rental[rental["return_date"].notna()].copy()
r["rental_date"] = pd.to_datetime(r["rental_date"])
r["return_date"] = pd.to_datetime(r["return_date"])
r["days"] = (r["return_date"] - r["rental_date"]).dt.total_seconds() / 86400.0

# 2) film bazında max süre
m_film = r.merge(inventory, on="inventory_id").merge(film, on="film_id")
film_dur = (m_film.groupby(["film_id","title"], as_index=False)
                   .agg(max_days=("days","max")))

mx = film_dur["max_days"].max()
longest = film_dur[film_dur["max_days"] == mx][["film_id","title","max_days"]]

# 3) mağaza bilgisi
pd_s63 = (longest.merge(inventory, on="film_id")
                   .merge(r, on="inventory_id")
                   .loc[:, ["film_id","title","max_days","store_id"]]
                   .drop_duplicates()
                   .sort_values("store_id")
                   .reset_index(drop=True))

pd_s63


,film_id,title,max_days,store_id
0,416,HIGHBALL POTTER,9.249306,1
1,424,HOLOCAUST HIGHBALL,9.249306,1
2,416,HIGHBALL POTTER,9.249306,2
3,424,HOLOCAUST HIGHBALL,9.249306,2


# Soru 65: Her filmin ortalama kiralama süresi nedir?

In [64]:
film      = pd.read_sql("SELECT film_id, title FROM film;", con)
inventory = pd.read_sql("SELECT inventory_id, film_id FROM inventory;", con)
rental    = pd.read_sql("SELECT inventory_id, rental_date, return_date FROM rental;", con)

# Yalnız iade edilmişler
r = rental[rental["return_date"].notna()].copy()
r["rental_date"] = pd.to_datetime(r["rental_date"])
r["return_date"] = pd.to_datetime(r["return_date"])
r["days"] = (r["return_date"] - r["rental_date"]).dt.total_seconds() / 86400.0

# Film bazında ortalama gün
m = r.merge(inventory, on="inventory_id").merge(film, on="film_id")
pd_s65 = (m.groupby(["film_id","title"], as_index=False)
           .agg(avg_rental_days=("days","mean"))
           .sort_values(["avg_rental_days","title"], ascending=[False, True])
           .reset_index(drop=True))
pd_s65


,film_id,title,avg_rental_days
0,323,FLIGHT LIES,7.270775
1,454,IMPACT ALADDIN,7.200617
2,5,AFRICAN EGG,7.106629
3,400,HARDLY ROBBERS,6.948438
4,546,MADRE GABLES,6.811806
...,...,...,...
953,885,TEXAS WATCH,3.016667
954,788,SHIP WONDERLAND,2.986296
955,335,FREEDOM CLEOPATRA,2.959722
956,477,JAWBREAKER BROOKLYN,2.585764


# Soru 66: Hangi filmler en çok kiralanan kategoride?

In [65]:
category      = pd.read_sql("SELECT category_id, name FROM category;", con)
film_category = pd.read_sql("SELECT film_id, category_id FROM film_category;", con)
film          = pd.read_sql("SELECT film_id, title FROM film;", con)
inventory     = pd.read_sql("SELECT inventory_id, film_id FROM inventory;", con)
rental        = pd.read_sql("SELECT rental_id, inventory_id FROM rental;", con)

# 1) kategori bazında toplam kiralama
m_cat = (category.merge(film_category, on="category_id")
                 .merge(film, on="film_id")
                 .merge(inventory, on="film_id")
                 .merge(rental, on="inventory_id"))

cat_cnt = (m_cat.groupby(["category_id","name"], as_index=False)
                 .agg(rental_count=("rental_id","count"))
                 .rename(columns={"name":"category_name"}))

# 2) en çok kiralanan kategori(ler)
mx = cat_cnt["rental_count"].max()
top_cat = cat_cnt[cat_cnt["rental_count"] == mx][["category_id","category_name"]]

# 3) bu kategorideki filmler ve kiralama sayıları
m_top = (top_cat.merge(film_category, on="category_id")
                  .merge(film, on="film_id")
                  .merge(inventory, on="film_id")
                  .merge(rental, on="inventory_id"))

pd_s66 = (m_top.groupby(["film_id","title","category_name"], as_index=False)
               .agg(film_rentals=("rental_id","count"))
               .sort_values(["film_rentals","title"], ascending=[False, True])
               .reset_index(drop=True))
pd_s66


,film_id,title,category_name,film_rentals
0,361,GLEAMING JAWBREAKER,Sports,29
1,875,TALENTED HOMICIDE,Sports,29
2,745,ROSES TREASURE,Sports,28
3,764,SATURDAY LAMBS,Sports,28
4,135,CHANCE RESURRECTION,Sports,27
...,...,...,...,...
68,779,SENSE GREEK,Sports,7
69,836,SQUAD FISH,Sports,7
70,102,BUBBLE GROSSE,Sports,6
71,372,GRACELAND DYNAMITE,Sports,6


# Soru 67: Hangi filmler en az kiralanan kategoride?

In [66]:
category      = pd.read_sql("SELECT category_id, name FROM category;", con)
film_category = pd.read_sql("SELECT film_id, category_id FROM film_category;", con)
film          = pd.read_sql("SELECT film_id, title FROM film;", con)
inventory     = pd.read_sql("SELECT inventory_id, film_id FROM inventory;", con)
rental        = pd.read_sql("SELECT rental_id, inventory_id FROM rental;", con)

# 1) kategori bazında toplam kiralama (LEFT JOIN mantığı: hiç kiralanmayanlar dahil)
m_cat = (category.merge(film_category, on="category_id")
                 .merge(film, on="film_id")
                 .merge(inventory, on="film_id")
                 .merge(rental, on="inventory_id", how="left"))

cat_cnt = (m_cat.groupby(["category_id","name"], as_index=False)
                 .agg(rental_count=("rental_id","count"))
                 .rename(columns={"name":"category_name"}))

# 2) en az kiralanan kategori(ler)
mn = cat_cnt["rental_count"].min()
min_cat = cat_cnt[cat_cnt["rental_count"] == mn][["category_id","category_name"]]

# 3) bu kategorideki filmler ve film bazında kiralanma sayıları
m_min = (min_cat.merge(film_category, on="category_id")
                  .merge(film, on="film_id")
                  .merge(inventory, on="film_id")
                  .merge(rental, on="inventory_id", how="left"))

pd_s67 = (m_min.groupby(["film_id","title","category_name"], as_index=False)
               .agg(film_rentals=("rental_id","count"))
               .sort_values(["film_rentals","title"], ascending=[True, True])
               .reset_index(drop=True))
pd_s67


,film_id,title,category_name,film_rentals
0,884,TERMINATOR CLUB,Music,6
1,997,YOUTH KICK,Music,6
2,161,CLUE GRAIL,Music,7
3,278,ELF MURDER,Music,7
4,411,HEAVENLY GUN,Music,7
5,384,GROSSE WONDERFUL,Music,8
6,718,REBEL AIRPORT,Music,8
7,752,RUNNER MADIGAN,Music,8
8,76,BIRDCAGE CASPER,Music,9
9,454,IMPACT ALADDIN,Music,9


# Soru 68: Hangi mağazalarda en çok film kiralanmış?

In [67]:
store     = pd.read_sql("SELECT store_id FROM store;", con)
inventory = pd.read_sql("SELECT inventory_id, store_id FROM inventory;", con)
rental    = pd.read_sql("SELECT rental_id, inventory_id FROM rental;", con)

# mağaza bazında kiralama sayısı
m = inventory.merge(rental, on="inventory_id")
store_cnt = (m.groupby("store_id", as_index=False)
               .agg(rental_count=("rental_id","count")))

# en yüksek değeri alan mağaza(lar)
mx = store_cnt["rental_count"].max()
pd_s68 = store_cnt[store_cnt["rental_count"] == mx] \
           .sort_values("store_id") \
           .reset_index(drop=True)

pd_s68  # store_id, rental_count


,store_id,rental_count
0,2,8121


# Soru 69: Hangi mağazalarda en az film kiralanmış?

In [68]:
store     = pd.read_sql("SELECT store_id FROM store;", con)
inventory = pd.read_sql("SELECT inventory_id, store_id FROM inventory;", con)
rental    = pd.read_sql("SELECT rental_id, inventory_id FROM rental;", con)

# mağaza bazında kiralama sayısı (LEFT mantığı: hiç kiralanmayan da sayılır)
m = inventory.merge(rental, on="inventory_id", how="left")
store_cnt = (m.groupby("store_id", as_index=False)
               .agg(rental_count=("rental_id","count")))

# en düşük değeri alan mağaza(lar)
mn = store_cnt["rental_count"].min()
pd_s69 = store_cnt[store_cnt["rental_count"] == mn] \
           .sort_values("store_id") \
           .reset_index(drop=True)

pd_s69  # store_id, rental_count


,store_id,rental_count
0,1,7923


# Soru 70: Hangi aktörler en çok filmde rol almış?

In [69]:
actor      = pd.read_sql("SELECT actor_id, first_name, last_name FROM actor;", con)
film_actor = pd.read_sql("SELECT film_id, actor_id FROM film_actor;", con)

# aktör bazında film sayısı
actor_cnt = (film_actor.groupby("actor_id", as_index=False)
                        .agg(film_count=("film_id","count"))
             .merge(actor, on="actor_id")
             .assign(actor_name=lambda d: d["first_name"]+" "+d["last_name"])
             .loc[:, ["actor_id","actor_name","film_count"]])

# en çok filmde oynayan(lar)
mx = actor_cnt["film_count"].max()
pd_s70 = actor_cnt[actor_cnt["film_count"] == mx] \
            .sort_values("actor_name") \
            .reset_index(drop=True)

pd_s70


,actor_id,actor_name,film_count
0,107,GINA DEGENERES,42


# Soru 71: Hangi aktörler en az filmde rol almış?

In [70]:
actor      = pd.read_sql("SELECT actor_id, first_name, last_name FROM actor;", con)
film_actor = pd.read_sql("SELECT film_id, actor_id FROM film_actor;", con)

# aktör bazında film sayısı (LEFT mantığı: hiç filmde oynamayan aktörleri de dahil et)
actor_cnt = (actor.merge(film_actor, on="actor_id", how="left")
                   .groupby(["actor_id","first_name","last_name"], as_index=False)
                   .agg(film_count=("film_id","count"))
                   .assign(actor_name=lambda d: d["first_name"]+" "+d["last_name"])
                   .loc[:, ["actor_id","actor_name","film_count"]])

# en az filmde oynayan(lar)
mn = actor_cnt["film_count"].min()
pd_s71 = actor_cnt[actor_cnt["film_count"] == mn] \
            .sort_values("actor_name") \
            .reset_index(drop=True)

pd_s71


,actor_id,actor_name,film_count
0,148,EMILY DEE,14


# Soru 72: Hangi aktörlerin oynadığı filmler en çok kiralanmış?

In [71]:
actor      = pd.read_sql("SELECT actor_id, first_name, last_name FROM actor;", con)
film_actor = pd.read_sql("SELECT film_id, actor_id FROM film_actor;", con)
inventory  = pd.read_sql("SELECT inventory_id, film_id FROM inventory;", con)
rental     = pd.read_sql("SELECT rental_id, inventory_id FROM rental;", con)

m = (film_actor
     .merge(inventory, on="film_id")
     .merge(rental, on="inventory_id"))

cnt = (m.groupby("actor_id", as_index=False)
         .agg(rental_count=("rental_id","count"))
         .merge(actor, on="actor_id")
         .assign(actor_name=lambda d: d["first_name"]+" "+d["last_name"])
         .loc[:, ["actor_id","actor_name","rental_count"]])

mx = cnt["rental_count"].max()
pd_s72 = (cnt[cnt["rental_count"] == mx]
          .sort_values("actor_name")
          .reset_index(drop=True))
pd_s72


,actor_id,actor_name,rental_count
0,107,GINA DEGENERES,753


# Soru 73: Hangi aktörlerin oynadığı filmler en az kiralanmış?

In [72]:
actor      = pd.read_sql("SELECT actor_id, first_name, last_name FROM actor;", con)
film_actor = pd.read_sql("SELECT film_id, actor_id FROM film_actor;", con)
inventory  = pd.read_sql("SELECT inventory_id, film_id FROM inventory;", con)
rental     = pd.read_sql("SELECT rental_id, inventory_id FROM rental;", con)

m = (actor
     .merge(film_actor, on="actor_id", how="left")
     .merge(inventory,  on="film_id",  how="left")
     .merge(rental,     on="inventory_id", how="left"))

cnt = (m.groupby("actor_id", as_index=False)
         .agg(rental_count=("rental_id","count"))
         .merge(actor, on="actor_id", how="left")
         .assign(actor_name=lambda d: d["first_name"]+" "+d["last_name"])
         .loc[:, ["actor_id","actor_name","rental_count"]])

mn = cnt["rental_count"].min()
pd_s73 = cnt[cnt["rental_count"] == mn] \
           .sort_values("actor_name") \
           .reset_index(drop=True)
pd_s73


,actor_id,actor_name,rental_count
0,148,EMILY DEE,216


# Soru 74: Hangi kategorilerde en fazla aktör oynamış?

In [73]:
category      = pd.read_sql("SELECT category_id, name FROM category;", con)
film_category = pd.read_sql("SELECT film_id, category_id FROM film_category;", con)
film_actor    = pd.read_sql("SELECT film_id, actor_id FROM film_actor;", con)

# kategori - film - aktör eşleşmeleri
m = (category.merge(film_category, on="category_id")
              .merge(film_actor, on="film_id"))

cat_actor = (m.groupby(["category_id","name"], as_index=False)
               .agg(actor_count=("actor_id","nunique"))
               .rename(columns={"name":"category_name"}))

mx = cat_actor["actor_count"].max()
pd_s74 = cat_actor[cat_actor["actor_count"] == mx] \
           .sort_values("category_name") \
           .reset_index(drop=True)

pd_s74


,category_id,category_name,actor_count
0,15,Sports,182


# Soru 75: Hangi kategorilerde en az aktör oynamış?

In [74]:
category      = pd.read_sql("SELECT category_id, name FROM category;", con)
film_category = pd.read_sql("SELECT film_id, category_id FROM film_category;", con)
film_actor    = pd.read_sql("SELECT film_id, actor_id FROM film_actor;", con)

m = (category
     .merge(film_category, on="category_id")
     .merge(film_actor, on="film_id", how="left"))   # LEFT: aktörsüz filmler dahil

cat_actor = (m.groupby(["category_id","name"], as_index=False)
               .agg(actor_count=("actor_id","nunique"))
               .rename(columns={"name":"category_name"}))

mn = cat_actor["actor_count"].min()
pd_s75 = (cat_actor[cat_actor["actor_count"] == mn]
          .sort_values("category_name")
          .reset_index(drop=True))
pd_s75


,category_id,category_name,actor_count
0,12,Music,144


# Soru 76: Hangi kategorilerde oynayan filmler en çok kiralanmış?

In [75]:
category      = pd.read_sql("SELECT category_id, name FROM category;", con)
film_category = pd.read_sql("SELECT film_id, category_id FROM film_category;", con)
film          = pd.read_sql("SELECT film_id FROM film;", con)
inventory     = pd.read_sql("SELECT inventory_id, film_id FROM inventory;", con)
rental        = pd.read_sql("SELECT rental_id, inventory_id FROM rental;", con)

# kategori - film - inventory - rental eşleşmesi
m = (category.merge(film_category, on="category_id")
               .merge(film, on="film_id")
               .merge(inventory, on="film_id")
               .merge(rental, on="inventory_id"))

cat_rentals = (m.groupby(["category_id","name"], as_index=False)
                 .agg(rental_count=("rental_id","count"))
                 .rename(columns={"name":"category_name"}))

mx = cat_rentals["rental_count"].max()
pd_s76 = (cat_rentals[cat_rentals["rental_count"] == mx]
          .sort_values("category_name")
          .reset_index(drop=True))

pd_s76


,category_id,category_name,rental_count
0,15,Sports,1179


# Soru 77: Hangi kategorilerde oynayan filmler en az kiralanmış?

In [76]:
category      = pd.read_sql("SELECT category_id, name FROM category;", con)
film_category = pd.read_sql("SELECT film_id, category_id FROM film_category;", con)
film          = pd.read_sql("SELECT film_id FROM film;", con)
inventory     = pd.read_sql("SELECT inventory_id, film_id FROM inventory;", con)
rental        = pd.read_sql("SELECT rental_id, inventory_id FROM rental;", con)

m = (category.merge(film_category, on="category_id")
               .merge(film, on="film_id")
               .merge(inventory, on="film_id")
               .merge(rental, on="inventory_id", how="left"))

cat_rentals = (m.groupby(["category_id","name"], as_index=False)
                 .agg(rental_count=("rental_id","count"))
                 .rename(columns={"name":"category_name"}))

mn = cat_rentals["rental_count"].min()
pd_s77 = (cat_rentals[cat_rentals["rental_count"] == mn]
          .sort_values("category_name")
          .reset_index(drop=True))
pd_s77


,category_id,category_name,rental_count
0,12,Music,830


# Soru 78: En fazla kazanç sağlayan kategoriler hangileri?

In [77]:
import sqlite3, pandas as pd
con = sqlite3.connect("sqlite-sakila.db")

category      = pd.read_sql("SELECT category_id, name FROM category;", con)
film_category = pd.read_sql("SELECT film_id, category_id FROM film_category;", con)
film          = pd.read_sql("SELECT film_id FROM film;", con)
inventory     = pd.read_sql("SELECT inventory_id, film_id FROM inventory;", con)
rental        = pd.read_sql("SELECT rental_id, inventory_id FROM rental;", con)
payment       = pd.read_sql("SELECT rental_id, amount FROM payment;", con)

m = (category.merge(film_category, on="category_id")
               .merge(film, on="film_id")
               .merge(inventory, on="film_id")
               .merge(rental, on="inventory_id")
               .merge(payment, on="rental_id"))

cat_rev = (m.groupby(["category_id","name"], as_index=False)
             .agg(total_revenue=("amount","sum"))
             .rename(columns={"name":"category_name"}))

mx = cat_rev["total_revenue"].max()
pd_s78 = (cat_rev[cat_rev["total_revenue"] == mx]
          .sort_values("category_name")
          .reset_index(drop=True))
pd_s78


,category_id,category_name,total_revenue
0,15,Sports,5314.21


# Soru 79: En az kazanç sağlayan kategoriler hangileri?

In [78]:
category      = pd.read_sql("SELECT category_id, name FROM category;", con)
film_category = pd.read_sql("SELECT film_id, category_id FROM film_category;", con)
film          = pd.read_sql("SELECT film_id FROM film;", con)
inventory     = pd.read_sql("SELECT inventory_id, film_id FROM inventory;", con)
rental        = pd.read_sql("SELECT rental_id, inventory_id FROM rental;", con)
payment       = pd.read_sql("SELECT rental_id, amount FROM payment;", con)

m = (category.merge(film_category, on="category_id")
               .merge(film, on="film_id")
               .merge(inventory, on="film_id")
               .merge(rental, on="inventory_id")
               .merge(payment, on="rental_id", how="left"))

cat_rev = (m.groupby(["category_id","name"], as_index=False)
             .agg(total_revenue=("amount", lambda x: x.fillna(0).sum()))
             .rename(columns={"name":"category_name"}))

mn = cat_rev["total_revenue"].min()
pd_s79 = (cat_rev[cat_rev["total_revenue"] == mn]
          .sort_values("category_name")
          .reset_index(drop=True))
pd_s79

,category_id,category_name,total_revenue
0,12,Music,3417.72


# Soru 80: En çok film kiralayan müşterilerin ortalama ödeme miktarı nedir?

In [79]:
rental   = pd.read_sql("SELECT rental_id, customer_id FROM rental;", con)
payment  = pd.read_sql("SELECT customer_id, amount FROM payment;", con)
customer = pd.read_sql("SELECT customer_id, first_name, last_name FROM customer;", con)

# 1) müşteri başına kiralama adedi
cnt = (rental.groupby("customer_id", as_index=False)
             .agg(total_rentals=("rental_id","count")))
mx = cnt["total_rentals"].max()
topc = cnt[cnt["total_rentals"] == mx][["customer_id","total_rentals"]]

# 2) bu müşterilerin ortalama ödeme miktarı
pd_s80 = (topc.merge(payment, on="customer_id")
               .groupby(["customer_id","total_rentals"], as_index=False)
               .agg(avg_payment=("amount","mean"))
               .merge(customer, on="customer_id", how="left")
               .assign(customer_name=lambda d: d["first_name"]+" "+d["last_name"])
               .loc[:, ["customer_id","customer_name","total_rentals","avg_payment"]]
               .sort_values("avg_payment", ascending=False)
               .reset_index(drop=True))
pd_s80


,customer_id,customer_name,total_rentals,avg_payment
0,148,ELEANOR HUNT,46,4.707391


# Soru 81: En az film kiralayan müşterilerin ortalama ödeme miktarı nedir?

In [80]:
customer = pd.read_sql("SELECT customer_id, first_name, last_name FROM customer;", con)
rental   = pd.read_sql("SELECT rental_id, customer_id FROM rental;", con)
payment  = pd.read_sql("SELECT customer_id, amount FROM payment;", con)

# tüm müşteriler için kiralama adedi (LEFT mantığı)
cnt = (customer.merge(rental, on="customer_id", how="left")
               .groupby(["customer_id","first_name","last_name"], as_index=False)
               .agg(total_rentals=("rental_id","count"))
               .assign(customer_name=lambda d: d["first_name"]+" "+d["last_name"])
               .loc[:, ["customer_id","customer_name","total_rentals"]])

# en az kiralayan(lar)
mn = cnt["total_rentals"].min()
minc = cnt[cnt["total_rentals"] == mn]

# bu müşterilerin ortalama ödeme miktarı (ödemesi yoksa NaN)
pd_s81 = (minc.merge(payment, on="customer_id", how="left")
               .groupby(["customer_id","customer_name","total_rentals"], as_index=False)
               .agg(avg_payment=("amount","mean"))
               .sort_values("avg_payment", ascending=True)
               .reset_index(drop=True))
pd_s81


,customer_id,customer_name,total_rentals,avg_payment
0,318,BRIAN WYMAN,12,4.406667


# Soru 82: Hangi mağazalarda hangi kategoriler en çok kiralanmış?

In [81]:
store        = pd.read_sql("SELECT store_id FROM store;", con)
inventory    = pd.read_sql("SELECT inventory_id, store_id, film_id FROM inventory;", con)
film         = pd.read_sql("SELECT film_id FROM film;", con)
film_category= pd.read_sql("SELECT film_id, category_id FROM film_category;", con)
category     = pd.read_sql("SELECT category_id, name FROM category;", con)
rental       = pd.read_sql("SELECT rental_id, inventory_id FROM rental;", con)

# mağaza - film - kategori - rental eşleşmesi
m = (store.merge(inventory, on="store_id")
            .merge(film, on="film_id")
            .merge(film_category, on="film_id")
            .merge(category, on="category_id")
            .merge(rental, on="inventory_id"))

store_cat = (m.groupby(["store_id","category_id","name"], as_index=False)
               .agg(rental_count=("rental_id","count"))
               .rename(columns={"name":"category_name"}))

# her mağaza için max kiralama
mx_store = store_cat.groupby("store_id")["rental_count"].transform("max")
pd_s82 = (store_cat[store_cat["rental_count"] == mx_store]
          .sort_values(["store_id","category_name"])
          .reset_index(drop=True))

pd_s82


,store_id,category_id,category_name,rental_count
0,1,1,Action,596
1,2,15,Sports,624


# Soru 83: Hangi mağazalarda hangi kategoriler en az kiralanmış?

In [82]:
store        = pd.read_sql("SELECT store_id FROM store;", con)
inventory    = pd.read_sql("SELECT inventory_id, store_id, film_id FROM inventory;", con)
film_category= pd.read_sql("SELECT film_id, category_id FROM film_category;", con)
category     = pd.read_sql("SELECT category_id, name FROM category;", con)
rental       = pd.read_sql("SELECT rental_id, inventory_id FROM rental;", con)

m = (store.merge(inventory, on="store_id")
            .merge(film_category, on="film_id")
            .merge(category, on="category_id")
            .merge(rental, on="inventory_id", how="left"))   # LEFT: 0 kiralama dahil

store_cat = (m.groupby(["store_id","category_id","name"], as_index=False)
               .agg(rental_count=("rental_id","count"))
               .rename(columns={"name":"category_name"}))

mn_store = store_cat.groupby("store_id")["rental_count"].transform("min")
pd_s83 = (store_cat[store_cat["rental_count"] == mn_store]
          .sort_values(["store_id","category_name"])
          .reset_index(drop=True))

pd_s83


,store_id,category_id,category_name,rental_count
0,1,11,Horror,386
1,2,12,Music,394


# Soru 84: En fazla sayıda film kiralayan mağaza hangisidir ve toplam kiralama sayısı nedir?

In [83]:
inventory = pd.read_sql("SELECT inventory_id, store_id FROM inventory;", con)
rental    = pd.read_sql("SELECT rental_id, inventory_id FROM rental;", con)

store_cnt = (inventory.merge(rental, on="inventory_id")
                      .groupby("store_id", as_index=False)
                      .agg(rental_count=("rental_id","count")))

mx = store_cnt["rental_count"].max()
pd_s84 = (store_cnt[store_cnt["rental_count"] == mx]
          .sort_values("store_id")
          .reset_index(drop=True))
pd_s84


,store_id,rental_count
0,2,8121


# Soru 85: En az sayıda film kiralayan mağaza hangisidir ve toplam kiralama sayısı nedir?

In [84]:
inventory = pd.read_sql("SELECT inventory_id, store_id FROM inventory;", con)
rental    = pd.read_sql("SELECT rental_id, inventory_id FROM rental;", con)

store_cnt = (inventory.merge(rental, on="inventory_id", how="left")
                      .groupby("store_id", as_index=False)
                      .agg(rental_count=("rental_id","count")))

mn = store_cnt["rental_count"].min()
pd_s85 = (store_cnt[store_cnt["rental_count"] == mn]
          .sort_values("store_id")
          .reset_index(drop=True))
pd_s85


,store_id,rental_count
0,1,7923


# Soru 86: En fazla sayıda kiralama yapan müşteri hangisidir ve toplam kiralama sayısı nedir?

In [85]:
customer = pd.read_sql("SELECT customer_id, first_name, last_name FROM customer;", con)
rental   = pd.read_sql("SELECT rental_id, customer_id FROM rental;", con)

cust_cnt = (rental.groupby("customer_id", as_index=False)
                   .agg(rental_count=("rental_id","count"))
                   .merge(customer, on="customer_id")
                   .assign(customer_name=lambda d: d["first_name"]+" "+d["last_name"])
                   .loc[:, ["customer_id","customer_name","rental_count"]])

mx = cust_cnt["rental_count"].max()
pd_s86 = (cust_cnt[cust_cnt["rental_count"] == mx]
          .sort_values("customer_name")
          .reset_index(drop=True))
pd_s86


,customer_id,customer_name,rental_count
0,148,ELEANOR HUNT,46


# Soru 87: En az sayıda kiralama yapan müşteri hangisidir ve toplam kiralama sayısı nedir?

In [86]:
customer = pd.read_sql("SELECT customer_id, first_name, last_name FROM customer;", con)
rental   = pd.read_sql("SELECT rental_id, customer_id FROM rental;", con)

cust_cnt = (customer.merge(rental, on="customer_id", how="left")
                    .groupby(["customer_id","first_name","last_name"], as_index=False)
                    .agg(rental_count=("rental_id","count"))
                    .assign(customer_name=lambda d: d["first_name"]+" "+d["last_name"])
                    .loc[:, ["customer_id","customer_name","rental_count"]])

mn = cust_cnt["rental_count"].min()
pd_s87 = (cust_cnt[cust_cnt["rental_count"] == mn]
          .sort_values("customer_name")
          .reset_index(drop=True))
pd_s87


,customer_id,customer_name,rental_count
0,318,BRIAN WYMAN,12


# Soru 88: En fazla sayıda kiralanan film hangisidir ve toplam kiralama sayısı nedir?

In [87]:
film      = pd.read_sql("SELECT film_id, title FROM film;", con)
inventory = pd.read_sql("SELECT inventory_id, film_id FROM inventory;", con)
rental    = pd.read_sql("SELECT rental_id, inventory_id FROM rental;", con)

film_cnt = (film.merge(inventory, on="film_id")
                 .merge(rental, on="inventory_id")
                 .groupby(["film_id","title"], as_index=False)
                 .agg(rental_count=("rental_id","count")))

mx = film_cnt["rental_count"].max()
pd_s88 = (film_cnt[film_cnt["rental_count"] == mx]
          .sort_values("title")
          .reset_index(drop=True))
pd_s88


,film_id,title,rental_count
0,103,BUCKET BROTHERHOOD,34


# Soru 89: En az sayıda kiralanan film hangisidir ve toplam kiralama sayısı nedir?

In [88]:
film      = pd.read_sql("SELECT film_id, title FROM film;", con)
inventory = pd.read_sql("SELECT inventory_id, film_id FROM inventory;", con)
rental    = pd.read_sql("SELECT rental_id, inventory_id FROM rental;", con)

film_cnt = (film.merge(inventory, on="film_id")
                 .merge(rental, on="inventory_id", how="left")
                 .groupby(["film_id","title"], as_index=False)
                 .agg(rental_count=("rental_id","count")))

mn = film_cnt["rental_count"].min()
pd_s89 = (film_cnt[film_cnt["rental_count"] == mn]
          .sort_values("title")
          .reset_index(drop=True))
pd_s89


,film_id,title,rental_count
0,400,HARDLY ROBBERS,4
1,584,MIXED DOORS,4
2,904,TRAIN BUNCH,4


# Soru 90: En fazla kazanç sağlayan müşteri hangisidir ve toplam kazancı nedir?

In [89]:
customer = pd.read_sql("SELECT customer_id, first_name, last_name FROM customer;", con)
payment  = pd.read_sql("SELECT customer_id, amount FROM payment;", con)

cust_rev = (payment.groupby("customer_id", as_index=False)
                    .agg(total_revenue=("amount","sum"))
            .merge(customer, on="customer_id", how="left")
            .assign(customer_name=lambda d: d["first_name"]+" "+d["last_name"])
            .loc[:, ["customer_id","customer_name","total_revenue"]])

mx = cust_rev["total_revenue"].max()
pd_s90 = (cust_rev[cust_rev["total_revenue"] == mx]
          .sort_values("customer_name")
          .reset_index(drop=True))
pd_s90


,customer_id,customer_name,total_revenue
0,526,KARL SEAL,221.55


# Soru 91: En az kazanç sağlayan müşteri hangisidir ve toplam kazancı nedir?


In [90]:
customer = pd.read_sql("SELECT customer_id, first_name, last_name FROM customer;", con)
payment  = pd.read_sql("SELECT customer_id, amount FROM payment;", con)

cust_rev = (customer.merge(payment, on="customer_id", how="left")
                     .groupby(["customer_id","first_name","last_name"], as_index=False)
                     .agg(total_revenue=("amount", lambda x: x.fillna(0).sum()))
                     .assign(customer_name=lambda d: d["first_name"]+" "+d["last_name"])
                     .loc[:, ["customer_id","customer_name","total_revenue"]])

mn = cust_rev["total_revenue"].min()
pd_s91 = (cust_rev[cust_rev["total_revenue"] == mn]
          .sort_values("customer_name")
          .reset_index(drop=True))
pd_s91


,customer_id,customer_name,total_revenue
0,248,CAROLINE BOWMAN,50.85


# Soru 92: Hangi film, hangi kategoride en çok kazanç sağlamış?

In [91]:
film          = pd.read_sql("SELECT film_id, title FROM film;", con)
film_category = pd.read_sql("SELECT film_id, category_id FROM film_category;", con)
category      = pd.read_sql("SELECT category_id, name FROM category;", con)
inventory     = pd.read_sql("SELECT inventory_id, film_id FROM inventory;", con)
rental        = pd.read_sql("SELECT rental_id, inventory_id FROM rental;", con)
payment       = pd.read_sql("SELECT rental_id, amount FROM payment;", con)

# 1) film - kategori - ödeme eşleşmesi
m = (film.merge(film_category, on="film_id")
          .merge(category, on="category_id")
          .merge(inventory, on="film_id")
          .merge(rental, on="inventory_id")
          .merge(payment, on="rental_id"))

# 2) kategori-film bazında toplam gelir
film_rev = (m.groupby(["film_id","title","category_id","name"], as_index=False)
              .agg(total_revenue=("amount","sum"))
              .rename(columns={"name":"category_name"}))

# 3) her kategori için max gelirli film(ler)
mx_per_cat = film_rev.groupby("category_id")["total_revenue"].transform("max")
pd_s92 = (film_rev[film_rev["total_revenue"] == mx_per_cat]
          .sort_values(["category_name","title"])
          .reset_index(drop=True))
pd_s92


,film_id,title,category_id,category_name,total_revenue
0,327,FOOL MOCKINGBIRD,1,Action,175.77
1,239,DOGMA FAMILY,2,Animation,178.70
2,48,BACKLASH UNDEFEATED,3,Children,158.81
3,843,STEEL SANTA,4,Classics,141.77
4,1000,ZORRO ARK,5,Comedy,214.69
5,973,WIFE TURN,6,Documentary,223.69
6,897,TORQUE BOUND,7,Drama,198.72
7,715,RANGE MOONWALKER,8,Family,179.73
8,460,INNOCENT USUAL,9,Foreign,191.74
9,563,MASSACRE USUAL,10,Games,179.70


# Soru 93: Hangi film, hangi kategoride en az kazanç sağlamış?


In [92]:
category      = pd.read_sql("SELECT category_id, name FROM category;", con)
film_category = pd.read_sql("SELECT film_id, category_id FROM film_category;", con)
film          = pd.read_sql("SELECT film_id, title FROM film;", con)
inventory     = pd.read_sql("SELECT inventory_id, film_id FROM inventory;", con)
rental        = pd.read_sql("SELECT rental_id, inventory_id FROM rental;", con)
payment       = pd.read_sql("SELECT rental_id, amount FROM payment;", con)

# kategori × film × ödeme eşleşmesi (LEFT mantığıyla 0 gelirler dahil)
m = (category.merge(film_category, on="category_id")
               .merge(film, on="film_id")
               .merge(inventory, on="film_id", how="left")
               .merge(rental, on="inventory_id", how="left")
               .merge(payment, on="rental_id", how="left"))

film_rev = (m.groupby(["category_id","name","film_id","title"], as_index=False)
              .agg(total_revenue=("amount", lambda s: s.fillna(0).sum()))
              .rename(columns={"name":"category_name"}))

mn_per_cat = film_rev.groupby("category_id")["total_revenue"].transform("min")
pd_s93 = (film_rev[film_rev["total_revenue"] == mn_per_cat]
          .sort_values(["category_name","title"])
          .reset_index(drop=True))

pd_s93  # columns: film_id, title, category_id, category_name, total_revenue


,category_id,category_name,film_id,title,total_revenue
0,1,Action,38,ARK RIDGEMONT,9.95
1,1,Action,318,FIREHOUSE VIETNAM,9.95
2,1,Action,802,SKY MIRACLE,9.95
3,2,Animation,36,ARGONAUTS TOWN,9.95
4,2,Animation,325,FLOATS GARDEN,9.95
5,3,Children,168,COMANCHEROS ENEMY,9.94
6,4,Classics,196,CRUELTY UNFORGIVEN,7.93
7,5,Comedy,335,FREEDOM CLEOPATRA,5.95
8,6,Documentary,261,DUFFEL APOCALYPSE,6.93
9,6,Documentary,996,YOUNG LANGUAGE,6.93


# Soru 94: En fazla kazanç sağlayan mağaza hangisidir ve toplam kazancı nedir?

In [93]:
inventory = pd.read_sql("SELECT inventory_id, store_id FROM inventory;", con)
rental    = pd.read_sql("SELECT rental_id, inventory_id FROM rental;", con)
payment   = pd.read_sql("SELECT rental_id, amount FROM payment;", con)

# mağaza × ödeme toplamı
m = (inventory.merge(rental, on="inventory_id")
               .merge(payment, on="rental_id"))

store_rev = (m.groupby("store_id", as_index=False)
               .agg(total_revenue=("amount","sum")))

mx = store_rev["total_revenue"].max()
pd_s94 = (store_rev[store_rev["total_revenue"] == mx]
          .sort_values("store_id")
          .reset_index(drop=True))
pd_s94


,store_id,total_revenue
0,2,33726.77


# Soru 95: En az kazanç sağlayan mağaza hangisidir ve toplam kazancı nedir?

In [94]:
inventory = pd.read_sql("SELECT inventory_id, store_id FROM inventory;", con)
rental    = pd.read_sql("SELECT rental_id, inventory_id FROM rental;", con)
payment   = pd.read_sql("SELECT rental_id, amount FROM payment;", con)

# mağaza × gelir (LEFT: hiç kiralama/ödeme olmasa da dahil)
m = (inventory.merge(rental, on="inventory_id", how="left")
               .merge(payment, on="rental_id", how="left"))

store_rev = (m.groupby("store_id", as_index=False)
               .agg(total_revenue=("amount", lambda s: s.fillna(0).sum())))

mn = store_rev["total_revenue"].min()
pd_s95 = (store_rev[store_rev["total_revenue"] == mn]
          .sort_values("store_id")
          .reset_index(drop=True))
pd_s95


,store_id,total_revenue
0,1,33679.79


# Soru 96: En fazla sayıda farklı film kiralayan müşteri hangisidir?

In [95]:
customer = pd.read_sql("SELECT customer_id, first_name, last_name FROM customer;", con)
rental   = pd.read_sql("SELECT rental_id, customer_id, inventory_id FROM rental;", con)
inventory= pd.read_sql("SELECT inventory_id, film_id FROM inventory;", con)

m = (rental.merge(inventory, on="inventory_id")
           .groupby("customer_id", as_index=False)
           .agg(distinct_films=("film_id","nunique"))
           .merge(customer, on="customer_id", how="left")
           .assign(customer_name=lambda d: d["first_name"]+" "+d["last_name"])
           .loc[:, ["customer_id","customer_name","distinct_films"]])

mx = m["distinct_films"].max()
pd_s96 = (m[m["distinct_films"] == mx]
          .sort_values("customer_name")
          .reset_index(drop=True))
pd_s96


,customer_id,customer_name,distinct_films
0,148,ELEANOR HUNT,46


# Soru 97: En az sayıda farklı film kiralayan müşteri hangisidir?

In [96]:
customer = pd.read_sql("SELECT customer_id, first_name, last_name FROM customer;", con)
rental   = pd.read_sql("SELECT rental_id, customer_id, inventory_id FROM rental;", con)
inventory= pd.read_sql("SELECT inventory_id, film_id FROM inventory;", con)

# LEFT mantığı: hiç kiralama yapmayan müşteriler de dahil
m = (customer.merge(rental, on="customer_id", how="left")
             .merge(inventory, on="inventory_id", how="left")
             .groupby(["customer_id","first_name","last_name"], as_index=False)
             .agg(distinct_films=("film_id", lambda x: x.nunique()))
             .assign(customer_name=lambda d: d["first_name"]+" "+d["last_name"])
             .loc[:, ["customer_id","customer_name","distinct_films"]])

mn = m["distinct_films"].min()
pd_s97 = (m[m["distinct_films"] == mn]
          .sort_values("customer_name")
          .reset_index(drop=True))
pd_s97


,customer_id,customer_name,distinct_films
0,318,BRIAN WYMAN,12


# Soru 98: En fazla sayıda farklı müşteri tarafından kiralanan film hangisidir?

In [97]:
film      = pd.read_sql("SELECT film_id, title FROM film;", con)
inventory = pd.read_sql("SELECT inventory_id, film_id FROM inventory;", con)
rental    = pd.read_sql("SELECT inventory_id, customer_id FROM rental;", con)

m = (film.merge(inventory, on="film_id")
          .merge(rental, on="inventory_id"))

film_cust = (m.groupby(["film_id","title"], as_index=False)
               .agg(distinct_customers=("customer_id","nunique")))

mx = film_cust["distinct_customers"].max()
pd_s98 = (film_cust[film_cust["distinct_customers"] == mx]
          .sort_values("title")
          .reset_index(drop=True))
pd_s98


,film_id,title,distinct_customers
0,103,BUCKET BROTHERHOOD,33


# Soru 99: En az sayıda farklı müşteri tarafından kiralanan film hangisidir?

In [98]:

film      = pd.read_sql("SELECT film_id, title FROM film;", con)
inventory = pd.read_sql("SELECT inventory_id, film_id FROM inventory;", con)
rental    = pd.read_sql("SELECT inventory_id, customer_id FROM rental;", con)

m = (film.merge(inventory, on="film_id", how="left")
          .merge(rental, on="inventory_id", how="left"))

film_cust = (m.groupby(["film_id","title"], as_index=False)
               .agg(distinct_customers=("customer_id", lambda x: x.nunique())))

mn = film_cust["distinct_customers"].min()
pd_s99 = (film_cust[film_cust["distinct_customers"] == mn]
          .sort_values("title")
          .reset_index(drop=True))
pd_s99


,film_id,title,distinct_customers
0,14,ALICE FANTASIA,0
1,33,APOLLO TEEN,0
2,36,ARGONAUTS TOWN,0
3,38,ARK RIDGEMONT,0
4,41,ARSENIC INDEPENDENCE,0
5,87,BOONDOCK BALLROOM,0
6,108,BUTCH PANTHER,0
7,128,CATCH AMISTAD,0
8,144,CHINATOWN GLADIATOR,0
9,148,CHOCOLATE DUCK,0


# Soru 100: Hangi mağazada, hangi kategoride en fazla kazanç sağlanmış?

In [99]:
store        = pd.read_sql("SELECT store_id FROM store;", con)
inventory    = pd.read_sql("SELECT inventory_id, store_id, film_id FROM inventory;", con)
film         = pd.read_sql("SELECT film_id FROM film;", con)
film_category= pd.read_sql("SELECT film_id, category_id FROM film_category;", con)
category     = pd.read_sql("SELECT category_id, name FROM category;", con)
rental       = pd.read_sql("SELECT rental_id, inventory_id FROM rental;", con)
payment      = pd.read_sql("SELECT rental_id, amount FROM payment;", con)

m = (store.merge(inventory, on="store_id")
           .merge(film, on="film_id")
           .merge(film_category, on="film_id")
           .merge(category, on="category_id")
           .merge(rental, on="inventory_id")
           .merge(payment, on="rental_id"))

scr = (m.groupby(["store_id","category_id","name"], as_index=False)
         .agg(total_revenue=("amount","sum"))
         .rename(columns={"name":"category_name"}))

mx = scr.groupby("store_id")["total_revenue"].transform("max")
pd_s100 = (scr[scr["total_revenue"] == mx]
           .sort_values(["store_id","category_name"])
           .reset_index(drop=True))
pd_s100


,store_id,category_id,category_name,total_revenue
0,1,7,Drama,2573.24
1,2,15,Sports,2825.75
